## Installation and Configuration

Install the following packages required to execute this notebook. 

use the latest major GA version of each package; i.e., --upgrade

In [49]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

! pip3 install --upgrade --quiet google-generativeai

! pip3 install --upgrade --quiet google-cloud-texttospeech

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)


In [2]:
PROJECT_ID = "slimeify"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "us-central1"  # @param {type: "string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [4]:
BUCKET_URI = f"gs://mlb-sluggers-bucket-0122-{PROJECT_ID}-unique"  # @param {type:"string"}

### **Only if your bucket doesn't already exist**
Run the following cell to create your Cloud Storage bucket.

In [5]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlb-sluggers-bucket-0122-slimeify-unique/...


### Set Gemini API Key

In [5]:
GEMINI_API_KEY="add-gemini_api-key"

### Import libraries

In [53]:
import google.generativeai as genai
import io
from google.cloud import aiplatform
from google.cloud import storage
from google.cloud import texttospeech
from vertexai.preview.vision_models import ImageGenerationModel

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [21]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### Configure genai

Configure genai with the GEMINI_API_KEY for your project.

In [22]:
genai.configure(api_key=GEMINI_API_KEY)

# Define Functions for Fetching Data from MLB 

Define game fetch, rephrase and main functions

In [23]:
import requests
import json
import pandas as pd
from IPython.display import HTML, Image

def fetch_single_game_data(game_pk):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    response = requests.get(url)
    return response.json()

def get_highly_captivating_plays(all_plays):
    """
    Filter plays to only include those with a captivation index greater than 0.
    
    Args:
        all_plays (list): List of play dictionaries from the MLB game data
        
    Returns:
        list: Filtered list of plays with captivation index > 0
    """
    return [
        play for play in all_plays 
        if play.get('about', {}).get('captivatingIndex', 0) > 0
    ]

def get_scoring_play_ids(game_data):
    """
    Extracts the list of playIds for all scoring plays from the game data.

    Parameters:
        game_data (dict): The game JSON object.

    Returns:
        list: List of playIds corresponding to scoring plays.
    """
    # Access scoringPlays and allPlays from the game data
    scoring_play_indices = game_data.get("liveData", {}).get("plays", {}).get("scoringPlays", [])
    all_plays = game_data.get("liveData", {}).get("plays", {}).get("allPlays", [])
    
    play_descriptions = []
    play_ids = []
    for at_bat_index in scoring_play_indices:
        if at_bat_index < len(all_plays):  # Ensure the index is within bounds
            play = all_plays[at_bat_index]
            play_description = play.get("result").get("description")
            if play_description:
                play_descriptions.append(play_description)
            # Extract playId from the playEvents node if present
            play_id = play.get("playEvents", [{}])[0].get("playId")
            if play_id:
                play_ids.append(play_id)

    display(play_descriptions)
    return play_ids

def display_scoring_play_videos(play_ids):
    single_play_video_url = f'https://www.mlb.com/video/jazz-chisholm-jr-homers-2-on-a-fly-ball-to-right-center-field?q=playid={play_ids[1]}'
    display(single_play_video_url)
    HTML(f"""<video width="640" height="360" controls>
          <source src="{single_play_video_url}" type="video/mp4">
          Your browser does not support the video tag.
        </video>""")

def fetch_game_data(game_pk):
    """
    Fetch play-by-play game data from MLB StatsAPI.

    Args:
        game_pk (str): The game primary key (ID).

    Returns:
        list: A list of play descriptions from the game.
    """
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    response = requests.get(url)

    if response.status_code == 200:
        game_data = response.json()
        plays = game_data.get('liveData', {}).get('plays', {}).get('allPlays', [])
        # print(f"first play in game_data is: {plays[0]}")
        descriptions = [play['result']['description'] for play in plays if 'description' in play['result']]
        return descriptions
    else:
        print(f"Error fetching game data: {response.status_code}")
        return []
    
def hydrate_game_highlights():
    url = 'https://statsapi.mlb.com/api/v1/schedule/?sportId=1&date=10/30/2024&hydrate=game(content(highlights(highlights)))'
    response = requests.get(url)
    display(response.json())
    
def fetch_hydrated_schedule_data(dateStr):
    """
    Fetch play-by-play game data from MLB StatsAPI.

    Args:
        game_pk (str): The game primary key (ID).

    Returns:
        list: A list of play descriptions from the game.
    """
    url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&season=2024&hydrate=game(content(highlights(scoreboard)))'
    response = requests.get(url)

    if response.status_code == 200:
        game_data = response.json()
        # plays = game_data.get('liveData', {}).get('plays', {}).get('allPlays', [])
        # print(f"response is: {game_data}")
        # descriptions = [play['result']['description'] for play in plays if 'description' in play['result']]
        return game_data
    else:
        print(f"Error fetching game data: {response.status_code}{response.reason}")
        return []
    
def fetch_scoring_play_ids(game_pk):
    """
    Fetch play-by-play game data from MLB StatsAPI.

    Args:
        game_pk (str): The game primary key (ID).

    Returns:
        list: A list of play descriptions from the game.
    """
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    response = requests.get(url)

    if response.status_code == 200:
        game_data = response.json()
        plays = game_data.get('liveData', {}).get('plays', {})
        plays_by_at_bat_index = plays.get('scoringPlays', [])
        # print(f"first play in game_data is: {plays[0]}")
        # descriptions = [play['result']['description'] for play in plays if 'description' in play['result']]
        all_plays = plays.get('allPlays', [])
        play_ids = [play['id'] for play in all_plays if plays_by_at_bat_index.contains(play['atBatIndex'])]
        return play_ids
    else:
        print(f"Error fetching game data: {response.status_code}")
        return []
    
# Extract play-by-play details
def extract_play_by_play(game_data):
    plays = game_data["liveData"]["plays"]["allPlays"]
    play_details = []
    for play in plays:
        result = play["result"]
        about = play["about"]
        match_up = play["matchup"]
        play_info = {
            "description": result["description"],
            "inning": about["inning"],
            "half": "Top" if about["isTopInning"] else "Bottom",
            "event": result["event"],
            "away_score": result["awayScore"],
            "home_score": result["homeScore"],
            "batter": match_up["batter"]["fullName"],
            "pitcher": match_up["pitcher"]["fullName"],
            "captivating_index": about["captivatingIndex"]
        }
        play_details.append(play_info)
    return play_details

# Extract game overview
def extract_game_overview(data):
    game_info = data["gameData"]
    overview = {
        "away_team": game_info["teams"]["away"]["name"],
        "home_team": game_info["teams"]["home"]["name"],
        "venue": game_info["venue"]["name"],
        "date": game_info["datetime"]["officialDate"],
        "attendance": game_info["gameInfo"].get("attendance", "Unknown"),
        "duration_minutes": game_info["gameInfo"].get("gameDurationMinutes", "Unknown"),
        "weather": game_info["weather"],
    }
    return overview

# Define Function to Define the Captivating Plays 

Return a list of the playids of the most captivation plays based on the captivating index values

In [ ]:
import numpy as np

def convert_to_hyphenated(text: str) -> str:
    """
    Convert a string to lowercase hyphenated format, replacing non-alphanumeric chars with hyphens.
    Preserves accented characters.
    
    Args:
        text: Input string to convert
        
    Returns:
        Hyphenated lowercase string with alphanumeric chars, accents, and hyphens
        
    Example:
        >>> convert_to_hyphenated("Enrique Hernández homers (3)!")
        'enrique-hernández-homers-3'
    """
    # Convert to lowercase
    text = text.lower()
    
    # Replace non-alphanumeric chars with hyphens, but preserve accented characters
    import re
    text = re.sub(r'[^a-zá-úñ0-9]+', '-', text)
    
    # Remove leading/trailing hyphens
    text = text.strip('-')
    
    return text

def select_captivating_plays(game_data, num_highlights=5):
    """
    Selects a significant number of the most captivating instances from game data.

    Args:
    data: A dictionary containing game data, including play-by-play information.
    num_highlights: The desired number of highlights to select.

    Returns:
    A list of MLB video URLs with formatted descriptions and play IDs.
    """
    all_plays = game_data['liveData']['plays']['allPlays']
    captivating_indices = [play['about']['captivatingIndex'] for play in all_plays if 'captivatingIndex' in play['about']]
    
    if not captivating_indices:
        return []
    average_captivating_index = np.mean(captivating_indices)
    
    if len(all_plays) > 25 and average_captivating_index < 20:        
        threshold = average_captivating_index + 20
    elif len(all_plays) > 25 and average_captivating_index < 40:
        threshold = average_captivating_index + 10
    else:
        threshold = average_captivating_index
    
    highlight_plays = [play for play in all_plays if 'captivatingIndex' in play['about'] and play['about']['captivatingIndex'] > threshold]
    highlight_plays.sort(key=lambda play: play['about']['captivatingIndex'], reverse=True)
    
    selected_plays = [
        f"https://www.mlb.com/video/{convert_to_hyphenated(play['result']['description'])}?q=playid={play['playEvents'][-1]['playId']}"
        for play in highlight_plays[:num_highlights] 
        if 'playId' in play['playEvents'][-1] and 'description' in play['result']
    ]
    return selected_plays

In [ ]:
game_data = fetch_single_game_data("775296")
display(select_captivating_plays(game_data, 20))

# Define Function for Rephrasing 

Define game fetch, rephrase and main functions

In [ ]:
def rephrase_with_gemini(text):
    """
    Rephrase text using Gemini API.

    Args:
        text (str): Text to be rephrased.

    Returns:
        str: Rephrased text.
    """
    
    model = genai.GenerativeModel(
            model_name="gemini-1.5-pro"
        )
    
    prompt = f"""Given this description of a play in a baseball game: {text}, generate a concise explanation of the baseball strategy or tactics behind the play. Focus only on information directly relevant to the play described, without inventing or inferring events not explicitly mentioned. Assume the reader has a basic knowledge of the game and its rules. Keep the explanation clear, engaging, and limited to the strategic or tactical elements involved. Generate only the explanation, without additional formatting or contextual information."""

    response = model.generate_content(prompt)

    if response:
        return response.text.strip()
    else:
        print(f"Error with Gemini API: {response.status_code}, {response.text}")
        return text

# Define Main Function

Define main to orchestrate the rephrashing operation and its output

In [ ]:
def main():
    # Define configurations
    game_pk = "775296"  # Example game ID
    project_id = PROJECT_ID
    location = REGION

    # Fetch game data
    play_descriptions = fetch_game_data(game_pk)
    print("Original Description:", play_descriptions[0])

    # Rephrase descriptions
    rephrased_descriptions = [
        rephrase_with_gemini(play_descriptions[0])
        # for description in play_descriptions
    ]
    print("Rephrased Description:", rephrased_descriptions[0])

    # Save results
#     results_df = pd.DataFrame({
#         "Original": play_descriptions,
#         "Rephrased": rephrased_descriptions
#     })
#     results_df.to_csv("rephrased_game_events.csv", index=False)

#     print("Rephrased descriptions saved to rephrased_game_events.csv")

## Execute the Main Function to Rephrase a Description

Run main() to generate the rephrased description of the first play in the list received from the api.

In [ ]:
if __name__ == "__main__":
    main()

# Run Endpoint Hydration tests

In [ ]:
game_data = fetch_single_game_data("775296")
highlight_plays = get_highly_captivating_plays(game_data['liveData']['plays']['allPlays'])
all_plays = game_data['liveData']['plays']['allPlays']
model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")
prompt = f"""
Given the provided {game_data} select a significant number of the most captivating instances from {all_plays} based on the captivatingIndex field.
Consider the following factors:
Prioritize plays with higher captivatingIndex values.
Determine the average captivatingIndex for all plays in the game.
If the average captivatingIndex is low, adjust the threshold for selecting highlights to include more plays.
Consider the overall game narrative and pacing when selecting highlights.
Output:
Return an array of selected play_ids.
"""
response = model.generate_content(prompt)

display(response.text.strip())
# play_ids = get_scoring_play_ids(game_data)
# display_scoring_play_videos(play_ids)
# hydrate_game_highlights()

# Define Functions to Create Stories and Storyboards

In [72]:
from typing import Optional, List
import typing_extensions as typing

class Scene(typing.TypedDict):
    actNumber: int                     # The act number of the scene
    sceneNumber: int                   # The scene number within the act
    description: str                   # A description of the scene
    imageUrl: Optional[str]            # URL to the scene's image (if available)
    audioUrl_en: Optional[str]         # URL to the English audio file (if available)
    audioUrl_es: Optional[str]         # URL to the Spanish audio file (if available)
    audioUrl_ja: Optional[str]         # URL to the Japanese audio file (if available)
    caption_en: Optional[str]          # English caption text for the scene
    caption_es: Optional[str]          # Spanish caption text for the scene
    caption_ja: Optional[str]          # Japanese caption text for the scene
    visualDescription: Optional[str]   # Brief text description of the scene
    imagenPrompt: Optional[str]        # Description used to generate an image (e.g., for AI rendering)


class Storyboard(typing.TypedDict):
    scenes: list[Scene]                 # List of scenes within the story

def tell_the_plays_as_a_story(text, model):
    prompt = f"""Given this JSON of a completed baseball game: {text}, tell the story of the game in a structured three-act format that is captivating for children. Focus only on information directly relevant to the plays described, without inventing or inferring events not explicitly represented in the data. Assume the reader has a basic knowledge of the game and its rules. Incorporate the following:
Three-Act Structure:
Opening Shot (Act 1, Scene 0): establish the story being told
Act 1 (Setup): Introduce the teams, key players, and the opening moments of the game. Build anticipation for the action to come.
Act 2 (Conflict): Highlight the tension and turning points in the game, focusing on suspenseful and strategic plays. Show the rise and fall of momentum between the teams.
Act 3 (Resolution): Conclude with the final dramatic moments of the game, including any climactic plays and the outcome. End with a sense of closure or excitement about the game's conclusion.
Closing Shot (Act 3 final scene): wrap up the story for the viewer
Each act should consist of 3-5 scenes depending on the {text} content.
The final story should be 11-17 scenes with opening and closing shots
Language Requirements:The story should be provided in English, Spanish, and Japanese. Each scene must include captions and audio URLs for all three languages.
Output the story in JSON format as an object of scenes following this schema:
For each scene, include the following:
actNumber: The act number for a given scene
sceneNumber: The sequential number of the scene in the entire story.
description: A brief description of what happens in the scene, emphasizing strategic or tactical elements.
imageUrl: Empty string as a placeholder
audioUrl_en: Empty string as a placeholder
audioUrl_es: Empty string as a placeholder
audioUrl_ja: Empty string as a placeholder
caption_en: A chunk of text in english derived from a larger text body called story.
caption_es: A chunk of text in spanish derived from a larger text body called story.
caption_ja: A chunk of text in japanese derived from a larger text body called story.
visualDescription: Brief Visual description of what happens in the scene, considering what a single still image would depict
imagenPrompt: Empty string as a placeholderUse clear and concise language that is easy for children to understand while incorporating elements like suspense, humor, and excitement to make the story captivating.
Output in JSON format, ensuring the structure of the JSON object remains consistent with the schema provided, is in valid JSON format, and ensuring that both key and string values use double quotes for all entries."""

    response = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json"
        ),
    )

    if response:
        return response.text.strip()
    else:
        print(f"Error with Gemini API: {response.status_code}, {response.text}")
        return text
    
def generate_storyboard_prompts(plays, story, model):
    prompt = f"""Given the JSON object {story} containing scenes from a baseball game, create an advanced prompt for generative AI using animal characters as the main subject for each scene. No humans should output from the generated prompt and any background people should be replaced with woodland animal characters. Please use any identified MLB teams in the prompt as appropriate. Consider the following in your response:
- 3D rendered woodland animal characters with a pixar animated kids film style look. Always refer to them as 'woodland animals' and never use the word 'creatures'
- Visual appeal: Focus on creating a visually engaging and exciting experience.
- Player-Specific Details: Include accurate details about players' positions, teams, and jersey numbers in the prompts. Do not use player names in the prompts. Ensure consistency in animal representation for specific players across all scenes (e.g., if a player is represented as a lion with jersey number 22, this representation must remain consistent throughout the story).
For each scene:
- Use the visualDescription, description and caption_en fields from the JSON object to inform the AI prompt, considering what a single still image of the scene would depict.
- describe the player's team, position and jersey number accurately without specifying their name.
- Incorporate relevant environmental details (e.g., stadium, field, crowd reactions) and tactical actions described in the scene.
-  create prompts that are optimized for generative ai image creation
- Populate the imagenPrompt field in the corresponding JSON object with the generated prompt. All other fields should remain unedited from the {story} object and populate the values from {story} to their corresponding
Output the updated JSON object, ensuring the structure of the JSON object remains consistent with the original schema provided, is in JSON format, and ensuring all keys and string values use double quotes."""
    
    response = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json"
        ),
    )
        
    if response:
        return response.text.strip()
    else:
        print(f"Error with Gemini API: {response.status_code}, {response.text}")
        return story
    
def combine_captions(storyboard: dict) -> str:
    """Combines all English captions from a storyboard into a single narrative string.
    
    Args:
        storyboard: A dictionary containing scenes with caption_en fields
        
    Returns:
        A string combining all English captions with proper spacing and flow
    """
    if not storyboard or 'scenes' not in storyboard:
        return ""
    
    # Extract all English captions and filter out None/empty values
    captions = [scene.get('caption_en', '') for scene in storyboard['scenes'] 
                if scene.get('caption_en')]
    
    # Join the captions with spaces between them
    return ' '.join(captions)


# Define Generative Models

In [60]:
base_model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp"
)
imagen3_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
tts_client = texttospeech.TextToSpeechClient()

# Define Functions to Upload Blob from Memory to Cloud

In [61]:
def upload_blob_from_stream(bucket_name, file_obj, destination_blob_name):
    """Uploads bytes from a stream or other file-like object to a blob."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The stream or file (file-like object) from which to read
    # import io
    # file_obj = io.BytesIO()
    # file_obj.write(b"This is test data.")

    # The desired name of the uploaded GCS object (blob)
    # destination_blob_name = "storage-object-name"

    # Construct a client-side representation of the blob.
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Rewind the stream to the beginning. This step can be omitted if the input
    # stream will always be at a correct position.
    file_obj.seek(0)

    # Upload data from the stream to your bucket.
    blob.upload_from_file(file_obj)

    print(
        f"Stream data uploaded to {destination_blob_name} in bucket {bucket_name}."
    )

def upload_blob_from_memory(bucket_name, contents, destination_blob_name):
    """Uploads a file to the bucket."""

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The contents to upload to the file
    # contents = "these are my contents"

    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)

    print(
        f"{destination_blob_name} with contents {contents} uploaded to {bucket_name}."
    )

# Define Functions to Generate Images

In [250]:
prompt = """
A 3D rendered, pixar animated kids film style, still image of a baseball field, focusing on the batter's box. At the plate is a woodland animal, a bear wearing the away team's jersey number 17, with a bat, ready to swing. Around the field are several other woodland animals in the home team’s uniforms, ready to field. In the background, more woodland animals fill the stands. The scene is bright and sunny.
"""

response = imagen3_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    aspect_ratio="16:9",
    safety_filter_level="block_few",
    person_generation="dont_allow",
)
    
images = response.images
for image in images:
    print(f"image is: {image}")
    image.save(BUCKET_URI)

# Definte Functions to Generate Speech

In [74]:
OUTPUT_FILE_NAME = "mlb-tts-002.mp3"

def synthesize_highlight_from_ssml(text):
    """Synthesizes speech from the input string of ssml.

    Note: ssml must be well-formed according to:
        https://www.w3.org/TR/speech-synthesis/
    """
    ssml = f"<speak>{text}</speak>"
    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(ssml=ssml)

    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Neural2-D",
        ssml_gender=texttospeech.SsmlVoiceGender.MALE
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        speaking_rate=1.15,
        pitch=0.5
    )

    response = client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )

    # Create BytesIO object for cloud storage upload
    file_obj = io.BytesIO()
    file_obj.write(response.audio_content)
    
    # Remove 'gs://' from the bucket name
    bucket_name = BUCKET_URI.replace('gs://', '')
    upload_blob_from_stream(bucket_name, file_obj, OUTPUT_FILE_NAME)

    # Optionally save to local file as well
    # with open("output.mp3", "wb") as out:
    #     out.write(response.audio_content)
    #     print('Audio content written to file "output.mp3"')

def synthesize_highlight_speech(text: str):
    client = texttospeech.TextToSpeechClient()
    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want returned
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # Perform the text-to-speech request on the text input with the selected
    # voice parameters and audio file type
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )
    file_obj = io.BytesIO()
    file_obj.write(response.audio_content)
    upload_blob_from_stream(BUCKET_URI, file_obj, OUTPUT_FILE_NAME)
    

# Execute Functions for Tests

In [75]:
game_data = fetch_single_game_data("775296")
plays = extract_play_by_play(game_data)
story = tell_the_plays_as_a_story(plays, base_model)
# display(f"The story output is:\n {story}\n")
storyboard_prompts = generate_storyboard_prompts(plays, story, base_model)
storyboard_as_json = json.loads(storyboard_prompts)
first_board = storyboard_as_json['scenes'][0]
first_caption = first_board['caption_en']
display(first_caption)
all_en_captions = combine_captions(storyboard_as_json)
display(all_en_captions)
synthesize_highlight_from_ssml(all_en_captions)

'The game is about to start! Shohei Ohtani steps up to bat against the mighty pitcher, Gerrit Cole. The crowd is roaring!'

"The game is about to start! Shohei Ohtani steps up to bat against the mighty pitcher, Gerrit Cole. The crowd is roaring! Mookie Betts and Freddie Freeman try to get on base but are quickly stopped by Gerrit Cole.  Then,  Gleyber Torres is out! Wow! Aaron Judge hits a home run, scoring Juan Soto! Then, Jazz Chisholm Jr. hits ANOTHER one! The home team takes the lead! Teoscar Hernández, Max Muncy and Tommy Edman swing and miss!  Gerrit Cole is on fire with the strikeouts! Anthony Volpe doubles, and then Alex Verdugo singles him home!  More walks put runners on base, and the home team scores again! Giancarlo Stanton blasts a huge home run!  The home team continues to add to their lead. Enrique Hernández gets a hit, Tommy Edman reaches base on an error, and then, Mookie Betts and Freddie Freeman get big hits to tie the game! Teoscar Hernández doubles, and then, Gavin Lux hits a sac fly to bring home another run!  The away team is now leading! Juan Soto and Aaron Judge try to get things go

Stream data uploaded to mlb-tts-002.mp3 in bucket mlb-sluggers-bucket-0122-slimeify-unique.


# Execute Imagen Function for Tests

In [251]:
# prompt = storyboard_as_json['scenes'][0]["imagenPrompt"]
prompt = "A 3D rendered, pixar animated kids film style image of a baseball game. A woodland animal, a beaver with jersey number 17 on the visiting team, stands at the batter's plate in a baseball stadium, while another woodland animal, a bear with jersey number 45, pitches for the home team. The background features other woodland animals filling the stadium stands. Both the batter and pitcher are in their team uniforms. A woodland animal umpire is visible behind the batter."
image = generate_image_from_prompt(prompt)
display(f"image is: {image}")

# Game Story Tests

### Game Story Test 13

The storyboard_prompts output is:
 ```json
{
  "story": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Shohei Ohtani and Mookie Betts try to get things started, but the pitcher is tough!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers stepped up to the plate first, with Shohei Ohtani facing off against the Yankees' Gerrit Cole. Ohtani flew out to center fielder Aaron Judge. Next, Mookie Betts grounded out to first baseman Anthony Rizzo. Freddie Freeman followed, flying out to left fielder Alex Verdugo.",
      "caption_es": "Los Dodgers tomaron el plato primero, con Shohei Ohtani enfrentándose a Gerrit Cole de los Yankees. Ohtani fue eliminado con un elevado al jardinero central Aaron Judge. Luego, Mookie Betts fue eliminado con un rodado al primera base Anthony Rizzo. Freddie Freeman siguió, siendo eliminado con un elevado al jardinero izquierdo Alex Verdugo.",
      "caption_ja": "試合開始！ドジャースはまず、大谷翔平がヤンキースのゲリット・コールと対決。大谷はセンターのアーロン・ジャッジにフライアウト。続いて、ムーキー・ベッツが一塁手のアンソニー・リゾにゴロアウト。フレディ・フリーマンも続き、レフトのアレックス・バードゥーゴにフライアウト。",
      "visualDescription": "A baseball field with Shohei Ohtani at bat, Gerrit Cole pitching, and Aaron Judge in center field.",
      "imagenPrompt": "3D rendered Pixar style animation of a baseball stadium. At bat is a powerful panda bear, Shohei Ohtani in Dodgers uniform #17, swinging a bat. Pitching is a grizzly bear, Gerrit Cole in Yankees uniform #45, throwing a fastball. In center field is a majestic eagle, Aaron Judge in Yankees uniform #99, ready to catch. The basepaths are made of packed dirt. In the stands surrounding the field are numerous woodland creatures cheering, all in a 3D animated style, Pixar-esque."
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The Yankees respond with POWER! Aaron Judge and Jazz Chisholm Jr. hit HUGE home runs!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Yankees came up to bat, and after a groundout and a walk, Aaron Judge blasted a home run, scoring himself and Juan Soto! Then, Jazz Chisholm Jr. followed with another home run!",
      "caption_es": "Los Yankees llegaron al bate, y después de un roletazo y una base por bolas, ¡Aaron Judge conectó un jonrón, anotándose a sí mismo y a Juan Soto! ¡Luego, Jazz Chisholm Jr. lo siguió con otro jonrón!",
      "caption_ja": "ヤンキースの攻撃、ゴロアウトとフォアボールの後、アーロン・ジャッジがホームランを放ち、自身とフアン・ソトが得点！その後、ジャズ・チザム・ジュニアがホームランで続きました！",
      "visualDescription": "Aaron Judge hitting a home run with Juan Soto running towards home plate, and Jazz Chisholm Jr. celebrating his own home run.",
      "imagenPrompt": "3D rendered Pixar style animation of Yankee stadium. A majestic eagle, Aaron Judge in Yankees uniform #99, hitting a massive home run. A sleek fox, Juan Soto, in Yankees uniform #22, is sprinting towards home plate with a determined look. In the background, a cheetah, Jazz Chisholm Jr., in Yankees uniform #2, leaps in the air with joy, celebrating his own home run. Woodland creatures fill the stands, wildly cheering, all rendered in a 3D animated style, Pixar-esque."
    },
    {
      "actNumber": 1,
      "sceneNumber": 3,
      "description": "The Yankees keep adding runs! A double and a single help them extend their lead. What will the Dodgers do?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "In the second inning, Anthony Volpe hit a double, and Alex Verdugo singled him in, adding another run for the Yankees!",
      "caption_es": "En la segunda entrada, Anthony Volpe bateó un doble, y Alex Verdugo lo impulsó con un sencillo, ¡agregando otra carrera para los Yankees!",
      "caption_ja": "2回、アンソニー・ボルペが二塁打を放ち、アンソニー・ボルペが二塁打を放ち、アレックス・バードゥーゴがシングルで彼を返し、ヤンキースにさらなる得点を追加しました！",
      "visualDescription": "Anthony Volpe sliding into second base with a double, followed by Alex Verdugo hitting a single to bring him home.",
      "imagenPrompt": "3D rendered Pixar style animation of a baseball field. A speedy rabbit, Anthony Volpe in Yankees uniform #11, is sliding into second base, kicking up dust. Behind him, a powerful bull, Alex Verdugo in Yankees uniform #24, is hitting a single towards right field. A squirrel umpire signals safe. Woodland creatures fill the stadium stands, some cheering, some looking concerned. The scene is vibrant and dynamic, rendered in a 3D animated style, Pixar-esque."
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "Giancarlo Stanton joins the home run party! The Yankees' lead grows even bigger. Can the Dodgers catch up?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Giancarlo Stanton hits a solo home run in the third inning! The Yankees are pulling ahead!",
      "caption_es": "¡Giancarlo Stanton conecta un jonrón solitario en la tercera entrada! ¡Los Yankees se están adelantando!",
      "caption_ja": "3回、ジャンカルロ・スタントンがソロホームラン！ヤンキースがリードを広げています！",
      "visualDescription": "Giancarlo Stanton rounding the bases after hitting a solo home run.",
      "imagenPrompt": "3D rendered Pixar style animation of a baseball stadium. A giant moose, Giancarlo Stanton in Yankees uniform #27, is rounding the bases with a triumphant grin after hitting a solo home run. The sky is bright blue. In the stands, woodland creatures cheer wildly, some waving tiny Yankee flags, all rendered in a 3D animated style, Pixar-esque."
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The Dodgers start to FIGHT BACK! Singles and a double bring runners home, tying the game!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "In the fifth inning, the Dodgers rallied! Mookie Betts and Freddie Freeman both singled, bringing in runs. Then, Teoscar Hernández doubled, tying the game!",
      "caption_es": "¡En la quinta entrada, los Dodgers se recuperaron! Mookie Betts y Freddie Freeman conectaron sencillos, impulsando carreras. Luego, Teoscar Hernández conectó un doble, ¡empatando el juego!",
      "caption_ja": "5回、ドジャースが反撃開始！ムーキー・ベッツとフレディ・フリーマンがシングルを放ち、得点を挙げました。そして、テオスカー・ヘルナンデスが二塁打を放ち、同点に追いつきました！",
      "visualDescription": "Mookie Betts, Freddie Freeman, and Teoscar Hernandez hitting singles and a double, with runners scoring.",
      "imagenPrompt": "3D rendered Pixar style animation of a baseball stadium. A quick rabbit, Mookie Betts in Dodgers uniform #50, is running toward first base after hitting a single. A strong ox, Freddie Freeman in Dodgers uniform #5, is running towards home plate, having also hit a single. A swift jaguar, Teoscar Hernandez, is rounding second after hitting a double. Dodgers fans comprised of woodland creatures cheer from the stands, all in a 3D animated style, Pixar-esque."
    },
    {
      "actNumber": 2,
      "sceneNumber": 6,
      "description": "The Yankees take the lead AGAIN! A sacrifice fly brings in another run. The tension is HIGH!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "In the sixth inning, Giancarlo Stanton hits a sacrifice fly, bringing in Juan Soto and putting the Yankees back in the lead!",
      "caption_es": "¡En la sexta entrada, Giancarlo Stanton conecta un elevado de sacrificio, impulsando a Juan Soto y devolviendo a los Yankees a la delantera!",
      "caption_ja": "6回、ジャンカルロ・スタントンが犠牲フライを放ち、フアン・ソトを返し、ヤンキースを再びリードさせました！",
      "visualDescription": "Giancarlo Stanton hitting a sacrifice fly with Juan Soto scoring.",
      "imagenPrompt": "3D rendered Pixar style animation of a baseball field. A giant moose, Giancarlo Stanton in Yankees uniform #27, is hitting a fly ball. A sleek fox, Juan Soto, in Yankees uniform #22, is sliding headfirst into home plate. The expression on Soto's face is one of intense determination. Tension is high on the field. Yankees fans (woodland creatures) roar in the stands, animated in a 3D style, Pixar-esque."
    },
    {
      "actNumber": 3,
      "sceneNumber": 7,
      "description": "The Dodgers answer RIGHT BACK! A single, errors, and sacrifice flies tie the game ONCE MORE!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "In the eighth, the Dodgers tied the game again! Singles by Enrique Hernández and Tommy Edman, followed by a sacrifice fly from Gavin Lux and Mookie Betts scores the go ahead run!",
      "caption_es": "¡En la octava, los Dodgers empataron el juego nuevamente! Sencillos de Enrique Hernández y Tommy Edman, seguidos de un elevado de sacrificio de Gavin Lux!",
      "caption_ja": "8回、ドジャースが再び同点に追いつきました！エンリケ・ヘルナンデスとトミー・エドマンのシングル、そしてギャビン・ラックスからの犠牲フライが続きました！",
      "visualDescription": "Enrique Hernández and Tommy Edman hitting singles, Gavin Lux hitting a sacrifice fly, and runners crossing home plate.",
      "imagenPrompt": "3D rendered Pixar style animation of a baseball field. A quick hummingbird, Enrique Hernandez, and a determined wolf, Tommy Edman are sprinting to first base, each having hit singles. A crafty badger, Gavin Lux is hitting a sacrifice fly towards center field. A rabbit Mookie Betts is scoring the go ahead run, a triumphant grin on his face. The stands are full of woodland creatures in dodgers gear cheering. A 3D animated style, Pixar-esque, brings the scene to life."
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "The Dodgers score one more, and they shut down the Yankees! The Dodgers win!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers shut down the Yankees in the bottom of the ninth. The Dodgers WIN!",
      "caption_es": "¡Los Dodgers silenciaron a los Yankees en la parte baja de la novena. ¡Los Dodgers GANAN!",
      "caption_ja": "ドジャースは9回裏にヤンキースを抑え込みました。ドジャースの勝利！",
      "visualDescription": "Dodgers players celebrating their victory on the field.",
      "imagenPrompt": "3D rendered Pixar style animation of Dodger Stadium. The entire Dodgers team, comprised of various woodland creatures, are jumping and cheering on the field, celebrating their victory. Streamers and confetti fill the air. The stadium is packed with cheering fans, also woodland creatures, waving Dodgers banners. A sense of pure joy and victory permeates the scene, all animated in a 3D style, Pixar-esque."
    }
  ]
}
```


### Game Story Test 12

The storyboard_prompts output is:
 ```json
{
  "story": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Ohtani, Betts, and Freeman try to score early, but the Yankees' pitcher, Cole, is too tough.  Three up, three down for the visiting team!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The big game is here!  The Dodgers face the Yankees, with stars like Shohei Ohtani and Aaron Judge ready to play. Ohtani flies out! Next Mookie Betts grounds out. Freddie Freeman can't get on base either!",
      "caption_es": "¡El gran juego está aquí! Los Dodgers se enfrentan a los Yankees, con estrellas como Shohei Ohtani y Aaron Judge listos para jugar. ¡Ohtani es eliminado con un elevado! Luego, Mookie Betts es out con un rodado. ¡Freddie Freeman tampoco puede llegar a la base!",
      "caption_ja": "注目の試合が始まる！ドジャース対ヤンキース、大谷翔平やアーロン・ジャッジなどのスター選手が出場。大谷はフライアウト！続くムーキー・ベッツはゴロアウト。フレディ・フリーマンも塁に出られず！",
      "imagenPrompt": "3D Pixar style animated kids film still. A majestic golden eagle wearing a Los Angeles Dodgers uniform with jersey number 17 (Ohtani) flies out at bat against a towering grizzly bear in a New York Yankees uniform with jersey number 45 (Cole) on the pitcher's mound. A nimble cheetah wearing a Dodgers uniform, jersey number 50 (Betts), grounds out toward a sly fox wearing a Yankees uniform. A strong bison wearing a Dodgers uniform, jersey number 5 (Freeman), can't reach first base. Background features a baseball stadium filled with cheering woodland creatures. Bright, vibrant colors, dynamic pose, intense action."
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The Yankees come up to bat and Torres grounds out, but Juan Soto walks! Then Aaron Judge hits a HUGE home run, bringing Soto home! Jazz Chisholm Jr. hits ANOTHER home run!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Yankees are up! Torres grounds out but, Juan Soto walks! Then, BAM! Aaron Judge hits a home run, and Soto scores.  The crowd goes wild! Then, Jazz Chisholm Jr. hits ANOTHER home run!",
      "caption_es": "¡Los Yankees al bate! Torres es out con un rodado, pero Juan Soto recibe base por bolas. ¡Luego, BAM! Aaron Judge batea un jonrón y Soto anota. ¡El público se vuelve loco! ¡Luego, Jazz Chisholm Jr. batea OTRO jonrón!",
      "caption_ja": "ヤンキースの攻撃！トーレスはゴロアウト、しかしフアン・ソトがフォアボール！そして、ドーン！アーロン・ジャッジがホームランを打ち、ソトが生還。観客は大興奮！さらに、ジャズ・チザムJr.がまたもやホームラン！",
      "imagenPrompt": "3D Pixar style animated kids film still. A playful raccoon wearing a New York Yankees uniform grounds out. A graceful deer, wearing the number 22 (Soto) for the Yankees, walks to first base. Suddenly, a gigantic Kodiak bear, wearing Yankees jersey number 99 (Judge) smashes a home run, carrying the deer (Soto) on his back across home plate. A energetic kangaroo wearing a Yankee's uniform, number 2 (Chisholm Jr), hits another home run! The woodland creature crowd roars in excitement in the stands. Vivid, colorful, explosive scene, extreme dynamic action, celebrating."
    },
    {
      "actNumber": 1,
      "sceneNumber": 3,
      "description": "Stanton and Rizzo are out, but the Yankees are already winning 3-0! Can the Dodgers catch up?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Giancarlo Stanton lines out, and Rizzo strikes out to end the inning. The Yankees are winning 3-0! What will the Dodgers do?",
      "caption_es": "Giancarlo Stanton es out con una línea, y Rizzo se poncha para terminar la entrada. ¡Los Yankees ganan 3-0! ¿Qué harán los Dodgers?",
      "caption_ja": "ジャンカルロ・スタントンはライナーアウト、リゾは三振でイニング終了。ヤンキースが3-0でリード！ドジャースはどうする？",
      "imagenPrompt": "3D Pixar style animated kids film still. A powerful gorilla, wearing a New York Yankees uniform with jersey number 27 (Stanton), hits a lineout to the outfield. A burly badger wearing a Yankees uniform, jersey number 48 (Rizzo) strikes out with a bewildered expression. The scoreboard in the background shows Yankees leading 3-0. Woodland creature fans cheer and celebrate, while Dodgers players with worried expressions huddle on their bench. Focus on dejection of Dodgers, Yankees joy. Vivid colors, dramatic lighting."
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "The Dodgers struggle in the second, striking out and flying out. The Yankees add another run with a double from Volpe and a single from Verdugo!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers can't score! Then, Anthony Volpe doubles! After that, Alex Verdugo singles and scores Volpe.  The Yankees lead 4-0!",
      "imagenPrompt": "3D Pixar style animated kids film still. The Dodgers, represented by various frustrated animals, fail to score. A swift red fox wearing jersey number 11 (Volpe) for the Yankees doubles into center field, standing proudly on second base. Next, a cunning wolf, Verdugo in number 24 with the Yankees, hits a single, allowing the fox to score a run. Scoreboard shows the Yankees leading 4-0. Jubilant Yankees fans celebrate. Dynamic composition, bright colors, showcasing speed and skill."
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The Yankees keep scoring! Stanton hits a home run in the third! The score is now 5-0.  But the Dodgers are not giving up!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Giancarlo Stanton hits a home run! The Yankees' lead increases to 5-0! The Dodgers need to score!",
      "caption_es": "¡Giancarlo Stanton batea un jonrón! ¡La ventaja de los Yankees aumenta a 5-0! ¡Los Dodgers necesitan anotar!",
      "caption_ja": "ジャンカルロ・スタントンがホームランを打った！ヤンキースのリードは5-0に拡大！ドジャースは得点する必要がある！",
      "imagenPrompt": "3D Pixar style animated kids film still. Giancarlo Stanton, a powerful gorilla in a New York Yankees uniform, jersey number 27, crushes a towering home run. The baseball soars high above the stadium. The scoreboard displays a 5-0 lead for the Yankees. The Dodgers animals on the field look determined and slightly anxious. The Yankees crowd, a diverse group of woodland animals, erupts in celebration. Dynamic action shot, focus on power and determination. Vibrant colors, dramatic lighting."
    },
    {
      "actNumber": 2,
      "sceneNumber": 6,
      "description": "The Dodgers finally get going in the fifth! They score FIVE runs thanks to hits from Hernandez, Betts, and Freeman, and a double from Hernandez! It's tied!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers fight back! Enrique Hernandez singles. Then, Mookie Betts and Freddie Freeman get singles, and Teoscar Hernandez doubles! The Dodgers tie the game 5-5!",
      "imagenPrompt": "3D Pixar style animated kids film still. A scrappy chihuahua (Enrique Hernandez) hits a single. A speedy cheetah (Mookie Betts) and a strong bison (Freddie Freeman) follow with singles. Then Teoscar Hernandez, represented by a mighty bear wearing jersey 3, slams a double off the outfield wall. The Dodgers team score five runs! The scoreboard now shows a tie game at 5-5! The woodland creatures in the Dodger's stands roar with excitement. Explosive action, focus on teamwork and energy, bright vivid colors."
    },
    {
      "actNumber": 2,
      "sceneNumber": 7,
      "description": "The Yankees score one more run in the sixth after a lot of walks and a sacrifice fly by Stanton! Now it's 6-5 for the home team!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Yankees get a run back!  Stanton hits a sacrifice fly to score Soto. Yankees lead 6-5.",
      "caption_es": "¡Los Yankees recuperan una carrera! Stanton batea un fly de sacrificio para que anote Soto. Los Yankees lideran 6-5.",
      "caption_ja": "ヤンキースが1点を追加！スタントンが犠牲フライを打ち、ソトが生還。ヤンキースが6-5でリード。",
      "imagenPrompt": "3D Pixar style animated kids film still. The Dodgers are playing defense, while the Yankees are taking the lead. The gorilla, Giancarlo Stanton number 27, is hitting a sacrifice fly. The deer, Juan Soto, is sprinting from third base to home base. The Yankees take the lead 6-5. Intense action in the field."
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "The Dodgers are back! They score two runs in the eighth after singles by Hernandez and Edman, and a sacrifice fly by Lux! The Dodgers now lead 7-6!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers take the lead! A series of singles and a sacrifice fly by Lux gives the Dodgers a 7-6 lead!",
      "caption_es": "¡Los Dodgers toman la delantera! ¡Una serie de sencillos y un fly de sacrificio de Lux le da a los Dodgers una ventaja de 7-6!",
      "caption_ja": "ドジャースがリードを奪う！一連のシングルヒットとラックスの犠牲フライで、ドジャースが7-6でリード！",
      "imagenPrompt": "3D Pixar style animated kids film still. Enrique Hernandez hits a single again and then Edman hits a single followed by Lux with a sacrifice fly to deep center. The Dodgers take the lead in the 8th. The score is 7-6. The dodgers are celebrating."
    },
    {
      "actNumber": 3,
      "sceneNumber": 9,
      "description": "The Yankees try to come back in the bottom of the eighth! Judge doubles, but the Dodgers pitcher strikes out Rizzo to end the inning!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Yankees threaten! Aaron Judge doubles! But the Dodgers pitcher strikes out Rizzo to end the inning!",
      "caption_es": "¡Los Yankees amenazan! ¡Aaron Judge batea un doble! ¡Pero el lanzador de los Dodgers poncha a Rizzo para terminar la entrada!",
      "caption_ja": "ヤンキースが反撃！アーロン・ジャッジがツーベースヒット！しかし、ドジャースのピッチャーがリゾを三振に打ち取ってイニング終了！",
      "imagenPrompt": "3D Pixar style animated kids film still. The Yankess are trying to come back in the 8th, but the dodger defense is too tough. Aaron Judge, the bear wearing jersey 99, hits a double. Then the Dodgers pitcher, a determined otter wearing number 61, strikes out Rizzo, a badger wearing jersey 48, to end the inning. Vivid and dynamic."
    },
    {
      "actNumber": 3,
      "sceneNumber": 10,
      "description": "It's the ninth inning! The Dodgers can't score, but can they hold the Yankees?  They do!  Verdugo strikes out to end the game!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers can't add to their lead. Then, the Yankees are up in the bottom of the 9th. Verdugo strikes out! The Dodgers win 7-6!",
      "caption_es": "Los Dodgers no pueden aumentar su ventaja. Luego, los Yankees están al bate en la parte baja de la novena. ¡Verdugo se poncha! ¡Los Dodgers ganan 7-6!",
      "caption_ja": "ドジャースはリードを広げられず。そして、ヤンキースが9回裏の攻撃。バードゥーゴが三振！ドジャースが7-6で勝利！",
      "imagenPrompt": "3D Pixar style animated kids film still. Bottom of the 9th inning. Alex Verdugo, the wolf wearing jersey 24, strikes out, with the Dodgers winning 7-6. The Dodger players, animals are cheering and jumping around in excitement. The woodland creature fans in the Dodger stands are going wild. A celebration of a close and exciting game. Close up on the Dodgers rejoicing."
    }
  ]
}
```


### Game Story Test 11

The story output is:
 ```json
{
  "scenes": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Ohtani, Betts, and Freeman try to score early but are stopped by Cole, the pitcher!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The crack of the bat, the roar of the crowd – it's time for baseball! Shohei Ohtani steps up to the plate, ready to start the game with a bang. But Gerrit Cole, the pitcher, is ready too! With a mighty swing, Ohtani flies out! Next, Mookie Betts tries to sneak a hit, but no luck, grounds out! Then, Freddie Freeman steps up to the plate, but flies out! Three outs, and the inning is over!",
      "caption_es": "¡El chasquido del bate, el rugido de la multitud, es hora de béisbol! Shohei Ohtani se acerca al plato, listo para comenzar el juego con una explosión. ¡Pero Gerrit Cole, el lanzador, también está listo! ¡Con un poderoso swing, Ohtani vuela hacia afuera! ¡Luego, Mookie Betts intenta colar un golpe, pero no hay suerte, conecta un rodado! ¡Entonces, Freddie Freeman se acerca al plato, pero vuela hacia afuera! ¡Tres outs y la entrada ha terminado!",
      "caption_ja": "バットの音、観衆の歓声 - 野球の時間です！大谷翔平が打席に立ち、試合を勢いよく始めようとしています。しかし、投手のゲリット・コールも準備万端！力強いスイングで、大谷はフライアウト！次に、ムーキー・ベッツがヒットを狙いますが、残念ながらゴロアウト！そして、フレディ・フリーマンが打席に立ちますが、フライアウト！３アウトで、イニング終了！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The home team explodes into action! Judge and Chisholm Jr. hit HUGE home runs, making the score 3-0!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Now it's the home team's turn! Torres grounds out, but then Juan Soto walks, followed by...AARON JUDGE! He smashes a HUGE home run! The crowd goes wild! And then, as if that weren't enough, Jazz Chisholm Jr. hits ANOTHER home run! Wow! The home team is winning 3-0!",
      "caption_es": "¡Ahora es el turno del equipo local! Torres conecta un rodado, pero luego Juan Soto camina, seguido de... ¡AARON JUDGE! ¡Él aplasta un ENORME jonrón! ¡La multitud se vuelve loca! Y luego, como si eso no fuera suficiente, Jazz Chisholm Jr. ¡conecta OTRO jonrón! ¡Guau! ¡El equipo local está ganando 3-0!",
      "caption_ja": "さあ、ホームチームの番です！トーレスはゴロアウト、しかしそのあと、フアン・ソトがフォアボール、そして…アーロン・ジャッジ！彼が特大ホームランを叩き込みました！観衆は熱狂！そして、それだけでは足りないかのように、ジャズ・チザム・ジュニアがもう一本ホームラン！すごい！ホームチームが３対０でリード！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 3,
      "description": "Volpe gets a double and scores! Verdugo adds another run! The home team extends their lead to 4-0.",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team isn't slowing down! Anthony Volpe hits a double! Then, Austin Wells grounds out, but Volpe gets to 3rd! Next, Alex Verdugo smacks a single, and Volpe scores! The score is now 4-0!",
      "caption_es": "¡El equipo local no está disminuyendo la velocidad! ¡Anthony Volpe conecta un doble! Luego, Austin Wells conecta un rodado, ¡pero Volpe llega a la tercera base! ¡Luego, Alex Verdugo golpea un sencillo y Volpe anota! ¡El marcador ahora es 4-0!",
      "caption_ja": "ホームチームは止まりません！アンソニー・ボルペがツーベースヒット！次に、オースティン・ウェルズがゴロアウト、しかしボルペは３塁へ！次に、アレックス・バードゥーゴがシングルヒット、そしてボルペが得点！スコアは４対０！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "Stanton hits a home run! It's now 5-0! Can the away team catch up?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team keeps the pressure on! Giancarlo Stanton hits a home run! The score is now 5-0! Will the away team be able to score?",
      "caption_es": "¡El equipo local sigue presionando! ¡Giancarlo Stanton conecta un jonrón! ¡El marcador ahora es 5-0! ¿Podrá el equipo visitante anotar?",
      "caption_ja": "ホームチームはプレッシャーをかけ続けます！ジャンカルロ・スタントンがホームラン！スコアは５対０！アウェイチームは得点できるでしょうか？",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The away team fights back! They score five runs in the 5th inning, tying the game!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Wait a minute! The away team isn't giving up! In the 5th inning, they start a rally! With hits from Hernández, Betts, and Freeman, and a double from Hernández, they score five runs! The game is tied 5-5!",
      "caption_es": "¡Espera un minuto! ¡El equipo visitante no se rinde! ¡En la quinta entrada, comienzan una remontada! ¡Con hits de Hernández, Betts y Freeman, y un doble de Hernández, anotan cinco carreras! ¡El juego está empatado 5-5!",
      "caption_ja": "ちょっと待って！アウェイチームは諦めません！５回、反撃開始！エルナンデス、ベッツ、フリーマンのヒット、そしてエルナンデスのツーベースで、５点を獲得！ゲームは５対５の同点！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 6,
      "description": "The home team scores a run! They regain the lead 6-5!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team answers! Stanton hits a sacrifice fly, bringing Soto home. The score is now 6-5!",
      "caption_es": "¡El equipo local responde! Stanton conecta un fly de sacrificio, trayendo a Soto a casa. ¡El marcador ahora es 6-5!",
      "caption_ja": "ホームチームが応えます！スタントンが犠牲フライを打ち、ソトをホームに迎え入れます。スコアは６対５！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 7,
      "description": "The away team scores two runs in the 8th, making it 7-6",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team won't give up. With strategic plays including singles, walks, and sac flies, they grab the lead again. It's 7-6 now.",
      "caption_es": "El equipo visitante no se rendirá. Con jugadas estratégicas que incluyen sencillos, caminatas y moscas de sacrificio, recuperan el liderazgo nuevamente. Ahora es 7-6.",
      "caption_ja": "アウェイチームは諦めません。シングルヒット、フォアボール、犠牲フライなど、戦略的なプレーで再びリードを奪います。今は7-6です。",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "The game ends! The away team wins 7-6!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "It's the bottom of the 9th. The home team needs a run to tie! But the pitcher is tough! He strikes out two batters, and the last batter grounds out. Game over! The away team wins 7-6!",
      "caption_es": "Es la parte baja de la novena. ¡El equipo local necesita una carrera para empatar! ¡Pero el lanzador es duro! Él poncha a dos bateadores, y el último bateador conecta un rodado. ¡Juego terminado! ¡El equipo visitante gana 7-6!",
      "caption_ja": "９回の裏です。ホームチームは同点にするために１点が必要です！しかし、ピッチャーは手強い！彼は２人の打者を三振させ、最後の打者はゴロアウト。試合終了！アウェイチームが７対６で勝利！",
      "imagenPrompt": ""
    }
  ]
}
```

The storyboard_prompts output is:
 ```json
{
  "scenes": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Ohtani, Betts, and Freeman try to score early but are stopped by Cole, the pitcher!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The crack of the bat, the roar of the crowd – it's time for baseball! Shohei Ohtani steps up to the plate, ready to start the game with a bang. But Gerrit Cole, the pitcher, is ready too! With a mighty swing, Ohtani flies out! Next, Mookie Betts tries to sneak a hit, but no luck, grounds out! Then, Freddie Freeman steps up to the plate, but flies out! Three outs, and the inning is over!",
      "caption_es": "¡El chasquido del bate, el rugido de la multitud, es hora de béisbol! Shohei Ohtani se acerca al plato, listo para comenzar el juego con una explosión. ¡Pero Gerrit Cole, el lanzador, también está listo! ¡Con un poderoso swing, Ohtani vuela hacia afuera! ¡Luego, Mookie Betts intenta colar un golpe, pero no hay suerte, conecta un rodado! ¡Entonces, Freddie Freeman se acerca al plato, pero vuela hacia afuera! ¡Tres outs y la entrada ha terminado!",
      "caption_ja": "バットの音、観衆の歓声 - 野球の時間です！大谷翔平が打席に立ち、試合を勢いよく始めようとしています。しかし、投手のゲリット・コールも準備万端！力強いスイングで、大谷はフライアウト！次に、ムーキー・ベッツがヒットを狙いますが、残念ながらゴロアウト！そして、フレディ・フリーマンが打席に立ちますが、フライアウト！３アウトで、イニング終了！",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: A baseball stadium filled with woodland animals as fans. On the mound, Gerrit Cole, a determined badger in a New York Yankees uniform with jersey number 45, winds up to pitch. At bat, Shohei Ohtani, a majestic lion in a Los Angeles Dodgers uniform with jersey number 17, swings mightily but misses. Mookie Betts, a swift fox in a Dodgers uniform with jersey number 50, waits on deck. Freddie Freeman, a strong bear also in a Dodgers uniform with jersey number 5, follows. The scene captures the tense energy of the opening of a baseball game, vivid colors, bright stadium lights, dynamic action."
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The home team explodes into action! Judge and Chisholm Jr. hit HUGE home runs, making the score 3-0!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Now it's the home team's turn! Torres grounds out, but then Juan Soto walks, followed by...AARON JUDGE! He smashes a HUGE home run! The crowd goes wild! And then, as if that weren't enough, Jazz Chisholm Jr. hits ANOTHER home run! Wow! The home team is winning 3-0!",
      "caption_es": "¡Ahora es el turno del equipo local! Torres conecta un rodado, pero luego Juan Soto camina, seguido de... ¡AARON JUDGE! ¡Él aplasta un ENORME jonrón! ¡La multitud se vuelve loca! Y luego, como si eso no fuera suficiente, Jazz Chisholm Jr. ¡conecta OTRO jonrón! ¡Guau! ¡El equipo local está ganando 3-0!",
      "caption_ja": "さあ、ホームチームの番です！トーレスはゴロアウト、しかしそのあと、フアン・ソトがフォアボール、そして…アーロン・ジャッジ！彼が特大ホームランを叩き込みました！観衆は熱狂！そして、それだけでは足りないかのように、ジャズ・チザム・ジュニアがもう一本ホームラン！すごい！ホームチームが３対０でリード！",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: A packed baseball stadium with excited woodland animal fans. Aaron Judge, a towering giraffe in a New York Yankees uniform with jersey number 99, has just connected with the ball, sending it soaring out of the park for a home run. Juan Soto, a crafty raccoon in a Yankees uniform with jersey number 22, celebrates on first base. In the dugout, Jazz Chisholm Jr., a zippy cheetah in a Miami Marlins uniform with jersey number 2, waits eagerly. Vivid colors, dynamic composition, celebratory atmosphere."
    },
    {
      "actNumber": 2,
      "sceneNumber": 3,
      "description": "Volpe gets a double and scores! Verdugo adds another run! The home team extends their lead to 4-0.",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team isn't slowing down! Anthony Volpe hits a double! Then, Austin Wells grounds out, but Volpe gets to 3rd! Next, Alex Verdugo smacks a single, and Volpe scores! The score is now 4-0!",
      "caption_es": "¡El equipo local no está disminuyendo la velocidad! ¡Anthony Volpe conecta un doble! Luego, Austin Wells conecta un rodado, ¡pero Volpe llega a la tercera base! ¡Luego, Alex Verdugo golpea un sencillo y Volpe anota! ¡El marcador ahora es 4-0!",
      "caption_ja": "ホームチームは止まりません！アンソニー・ボルペがツーベースヒット！次に、オースティン・ウェルズがゴロアウト、しかしボルペは３塁へ！次に、アレックス・バードゥーゴがシングルヒット、そしてボルペが得点！スコアは４対０！",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: Anthony Volpe, a nimble squirrel in a New York Yankees uniform with jersey number 17, slides into home plate, scoring a run. Alex Verdugo, a stoic tortoise in a Yankees uniform with jersey number 24, watches from first base, having just hit a single. Austin Wells, an owl in a Yankees uniform, looks on with anticipation. The stadium is filled with cheering woodland creatures. The scoreboard displays 4-0. Dynamic action, vibrant colors, cheering crowd of animals."
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "Stanton hits a home run! It's now 5-0! Can the away team catch up?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team keeps the pressure on! Giancarlo Stanton hits a home run! The score is now 5-0! Will the away team be able to score?",
      "caption_es": "¡El equipo local sigue presionando! ¡Giancarlo Stanton conecta un jonrón! ¡El marcador ahora es 5-0! ¿Podrá el equipo visitante anotar?",
      "caption_ja": "ホームチームはプレッシャーをかけ続けます！ジャンカルロ・スタントンがホームラン！スコアは５対０！アウェイチームは得点できるでしょうか？",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: Giancarlo Stanton, a powerful gorilla in a New York Yankees uniform with jersey number 27, watches the baseball soar into the stands after hitting a home run. The stadium is packed with cheering woodland animals, bathed in the warm glow of stadium lights. Scoreboard displays 5-0. Dynamic, powerful pose, celebratory atmosphere."
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The away team fights back! They score five runs in the 5th inning, tying the game!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Wait a minute! The away team isn't giving up! In the 5th inning, they start a rally! With hits from Hernández, Betts, and Freeman, and a double from Hernández, they score five runs! The game is tied 5-5!",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: A dynamic baseball scene showing the away team's comeback. Hernández, represented by a leaping jackrabbit in a Los Angeles Dodgers uniform with jersey number 14, celebrates a double, while Mookie Betts, the swift fox in a Dodgers uniform, runs towards home base. Freddie Freeman, the strong bear in a Dodgers uniform, cheers them on. The scoreboard shows a tied game, 5-5. A mix of excited and surprised woodland animal fans fill the stadium."
    },
    {
      "actNumber": 3,
      "sceneNumber": 6,
      "description": "The home team scores a run! They regain the lead 6-5!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team answers! Stanton hits a sacrifice fly, bringing Soto home. The score is now 6-5!",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: Juan Soto, the crafty raccoon in a New York Yankees uniform with jersey number 22, slides safely into home plate as Giancarlo Stanton, the powerful gorilla in the same uniform, watches from first base, having just hit a sacrifice fly. A few woodland animals are visibly upset in the opposing dugout, but many Yankees fans are cheering wildly in the stands. The scoreboard reads 6-5."
    },
    {
      "actNumber": 3,
      "sceneNumber": 7,
      "description": "The away team scores two runs in the 8th, making it 7-6",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team won't give up. With strategic plays including singles, walks, and sac flies, they grab the lead again. It's 7-6 now.",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: Showing a tense baseball game in the 8th inning. A determined Hernandez, the leaping jackrabbit in Dodgers uniform, is running to second after strategic plays. The scoreboard reflects the score of 7-6, with the away team in the lead. Cheering Dodgers fans in the crowd."
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "The game ends! The away team wins 7-6!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "It's the bottom of the 9th. The home team needs a run to tie! But the pitcher is tough! He strikes out two batters, and the last batter grounds out. Game over! The away team wins 7-6!",
      "imagenPrompt": "3D render, Pixar style, animated kids film look: The final scene of the baseball game. The badger, Gerrit Cole, stands triumphantly on the mound in his Yankees uniform, while a disconsolate giraffe (Aaron Judge) stands on the side, having just grounded out. The scoreboard shows 7-6. The visiting woodland animal fans are celebrating wildly, while the home fans look dejected."
    }
  ]
}
```


### Game Story Test 10

The story output is:
```json
{
  "scenes": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Star players like Shohei Ohtani and Aaron Judge step up to the plate. It\'s a sunny day for baseball!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The game is on! Ohtani flies out, and Betts grounds out. Freeman also flies out. The top of the first ends quickly.",
      "caption_es": "¡El juego ha comenzado! Ohtani batea un elevado y Betts conecta un roletazo. Freeman también es eliminado con un elevado. La parte alta de la primera entrada termina rápido.",
      "caption_ja": "試合開始！大谷がフライアウト、ベッツがゴロアウト。フリーマンもフライアウト。初回表はあっという間に終了。",
      "imagenPrompt": ""
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The home team gets ready to bat! Will they score in the first inning?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Torres grounds out, but Soto walks! Then, Aaron Judge hits a HUGE home run! Soto scores too!",
      "caption_es": "¡Torres es eliminado con un roletazo, pero Soto recibe una base por bolas! ¡Luego, Aaron Judge conecta un ENORME jonrón! ¡Soto también anota!",
      "caption_ja": "トーレスがゴロアウト、しかしソトがフォアボール！そして、アーロン・ジャッジが特大ホームラン！ソトも得点！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 1,
      "sceneNumber": 3,
      "description": "WOW! Another home run! The home team is off to a great start!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Chisholm Jr. hits ANOTHER home run! Stanton lines out and Rizzo strikes out to end the inning.",
      "caption_es": "¡Chisholm Jr. conecta OTRO jonrón! Stanton es eliminado con una línea y Rizzo se poncha para terminar la entrada.",
      "caption_ja": "チゾムJr.がまたホームラン！スタントンがラインアウト、リゾが三振でイニング終了。",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "The away team tries to catch up! Will they be able to score against Gerrit Cole?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Hernández, Muncy, and Hernández strikeout or flyout. The away team is struggling to get on base",
      "caption_es": "Hernández, Muncy y Hernández son ponchados o eliminados con elevados. El equipo visitante está luchando por embasarse.",
      "caption_ja": "ヘルナンデス、マンシー、ヘルナンデスは三振かフライアウト。アウェイチームは出塁に苦戦。",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The home team keeps the pressure on! More runs are scored!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Volpe doubles, then scores on a single by Verdugo! Walks make the bases loaded, but only one run scores.",
      "caption_es": "¡Volpe conecta un doble y luego anota con un sencillo de Verdugo! Las bases por bolas llenan las bases, pero solo se anota una carrera.",
      "caption_ja": "ボルペが二塁打、そしてバードゥーゴのシングルで得点！フォアボールで満塁になるも、１点止まり。",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 6,
      "description": "The other team has a big inning, tying up the score",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team finally rallies! Singles and a double bring everyone home, tying the game 5-5!",
      "caption_es": "¡El equipo visitante finalmente se recupera! ¡Sencillos y un doble llevan a todos a casa, empatando el juego 5-5!",
      "caption_ja": "アウェイチームがようやく反撃！シングルとツーベースで一気に得点し、5対5の同点に！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 7,
      "description": "The home team gets one run back. It\'s a close game!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Stanton hits a sacrifice fly to bring Soto home. The home team goes up 6-5",
      "caption_es": "Stanton batea un elevado de sacrificio para traer a Soto a casa. El equipo local se adelanta 6-5",
      "caption_ja": "スタントンが犠牲フライを打ち、ソトが生還。地元チームが6-5とリード。",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "The away team gets another run. It\'s tied once again!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team fights back! With some lucky plays they score to make it 6-6!",
      "caption_es": "¡El equipo visitante contraataca! ¡Con algunas jugadas afortunadas, anotan para poner el marcador 6-6!",
      "caption_ja": "アウェイチームが反撃！いくつかの幸運なプレーで得点し、6-6に！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 9,
      "description": "The away team takes the lead!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team scores again, the score is now 7-6!",
      "caption_es": "El equipo visitante anota de nuevo, ¡el marcador ahora es 7-6!",
      "caption_ja": "アウェイチームが再び得点し、スコアは7-6に！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 10,
      "description": "Final chance for the home team! Can they pull off a win?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team can\'t score. The away team wins 7-6",
      "caption_es": "El equipo local no puede anotar. El equipo visitante gana 7-6",
      "caption_ja": "ホームチームは得点できず。アウェイチームが7-6で勝利",
      "imagenPrompt": ""
    }
  ]
}
```

The storyboard_prompts output is:
```json
{
  "scenes": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Star players like Shohei Ohtani and Aaron Judge step up to the plate. It\'s a sunny day for baseball!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The game is on! Ohtani flies out, and Betts grounds out. Freeman also flies out. The top of the first ends quickly.",
      "caption_es": "¡El juego ha comenzado! Ohtani batea un elevado y Betts conecta un roletazo. Freeman también es eliminado con un elevado. La parte alta de la primera entrada termina rápido.",
      "caption_ja": "試合開始！大谷がフライアウト、ベッツがゴロアウト。フリーマンもフライアウト。初回表はあっという間に終了。",
      "imagenPrompt": "3D render, Pixar style animated kids film. A baseball stadium on a sunny day. Shohei Ohtani, a stoic bison wearing a Los Angeles Angels jersey (number 17), flies out to center field. Mookie Betts, a nimble squirrel wearing a Los Angeles Dodgers jersey (number 50), grounds out to second base. Freddie Freeman, a gentle giant bear wearing a Los Angeles Dodgers jersey (number 5), flies out to left field. The opposing team, comprised of various woodland creatures, enthusiastically cheers. Background characters are also woodland animals. Wide-angle lens, dynamic composition, vibrant colors."
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The home team gets ready to bat! Will they score in the first inning?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Torres grounds out, but Soto walks! Then, Aaron Judge hits a HUGE home run! Soto scores too!",
      "caption_es": "¡Torres es eliminado con un roletazo, pero Soto recibe una base por bolas! ¡Luego, Aaron Judge conecta un ENORME jonrón! ¡Soto también anota!",
      "caption_ja": "トーレスがゴロアウト、しかしソトがフォアボール！そして、アーロン・ジャッジが特大ホームラン！ソトも得点！",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, bright daylight. Gleyber Torres, a playful fox wearing a New York Yankees jersey (number 25), grounds out. Juan Soto, a graceful swan wearing a New York Yankees jersey (number 22), walks to first base. Aaron Judge, a majestic lion wearing a New York Yankees jersey (number 99), hits a towering home run, carrying Soto on his back. The cheering crowd is comprised of diverse woodland creatures. Low-angle shot, dramatic lighting, focus on Judge\'s power. Background characters are also woodland animals."
    },
    {
      "actNumber": 1,
      "sceneNumber": 3,
      "description": "WOW! Another home run! The home team is off to a great start!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Chisholm Jr. hits ANOTHER home run! Stanton lines out and Rizzo strikes out to end the inning.",
      "caption_es": "¡Chisholm Jr. conecta OTRO jonrón! Stanton es eliminado con una línea y Rizzo se poncha para terminar la entrada.",
      "caption_ja": "チゾムJr.がまたホームラン！スタントンがラインアウト、リゾが三振でイニング終了。",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, late afternoon. Jazz Chisholm Jr., a speedy cheetah wearing a Miami Marlins jersey (number 2), hits a powerful home run. Giancarlo Stanton, a towering giraffe wearing a New York Yankees jersey (number 27), lines out to center field. Anthony Rizzo, a wise owl wearing a New York Yankees jersey (number 48), strikes out. Background characters are also woodland animals. Close-up on Chisholm\'s powerful swing, dynamic pose, warm lighting. Background characters are also woodland animals."
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "The away team tries to catch up! Will they be able to score against Gerrit Cole?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Hernández, Muncy, and Hernández strikeout or flyout. The away team is struggling to get on base",
      "caption_es": "Hernández, Muncy y Hernández son ponchados o eliminados con elevados. El equipo visitante está luchando por embasarse.",
      "caption_ja": "ヘルナンデス、マンシー、ヘルナンデスは三振かフライアウト。アウェイチームは出塁に苦戦。",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, dusk. Teoscar Hernández, a determined wolf wearing a Los Angeles Dodgers jersey (number 3), strikes out against Gerrit Cole, a powerful gorilla wearing a New York Yankees jersey (number 45), on the mound. Max Muncy, a sturdy badger wearing a Los Angeles Dodgers jersey (number 13), flies out to the outfield. Kike Hernández, a sly raccoon wearing a Los Angeles Dodgers jersey (number 8), pops out. Background characters are also woodland animals. Dramatic shadows, tense atmosphere, close-up on Gerrit Cole\'s intense focus. Background characters are also woodland animals."
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The home team keeps the pressure on! More runs are scored!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Volpe doubles, then scores on a single by Verdugo! Walks make the bases loaded, but only one run scores.",
      "caption_es": "¡Volpe conecta un doble y luego anota con un sencillo de Verdugo! Las bases por bolas llenan las bases, pero solo se anota una carrera.",
      "caption_ja": "ボルペが二塁打、そしてバードゥーゴのシングルで得点！フォアボールで満塁になるも、１点止まり。",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, night. Anthony Volpe, a swift deer wearing a New York Yankees jersey (number 11), slides into second base for a double. Alex Verdugo, a determined bull wearing a New York Yankees jersey (number 24), hits a single, bringing Volpe home. The bases are loaded with woodland animal players. Background characters are also woodland animals. Action shot, vibrant stadium lights, emphasis on Volpe\'s slide. Background characters are also woodland animals."
    },
    {
      "actNumber": 2,
      "sceneNumber": 6,
      "description": "The other team has a big inning, tying up the score",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team finally rallies! Singles and a double bring everyone home, tying the game 5-5!",
      "caption_es": "¡El equipo visitante finalmente se recupera! ¡Sencillos y un doble llevan a todos a casa, empatando el juego 5-5!",
      "caption_ja": "アウェイチームがようやく反撃！シングルとツーベースで一気に得点し、5対5の同点に！",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, night. A diverse group of woodland creatures, representing the Los Angeles Dodgers, celebrate scoring runs. One squirrel slides safely into home. Several single hits from different characters are shown as well as one double. Scoreboard shows 5-5. Background characters are also woodland animals. Energetic scene, close-up shots of the woodland creatures with varied facial expressions, warm and vibrant lighting, dynamic composition. Background characters are also woodland animals."
    },
    {
      "actNumber": 2,
      "sceneNumber": 7,
      "description": "The home team gets one run back. It\'s a close game!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Stanton hits a sacrifice fly to bring Soto home. The home team goes up 6-5",
      "caption_es": "Stanton batea un elevado de sacrificio para traer a Soto a casa. El equipo local se adelanta 6-5",
      "caption_ja": "スタントンが犠牲フライを打ち、ソトが生還。地元チームが6-5とリード。",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, night. Giancarlo Stanton, a towering giraffe wearing a New York Yankees jersey (number 27), hits a sacrifice fly. Juan Soto, a graceful swan wearing a New York Yankees jersey (number 22), sprints towards home plate to score. Scoreboard shows 6-5. Woodland animal fans celebrate in the stands. Wide-angle lens, focus on the action, dynamic pose, emotional expression, and vibrant lighting. Background characters are also woodland animals."
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "The away team gets another run. It\'s tied once again!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team fights back! With some lucky plays they score to make it 6-6!",
      "caption_es": "¡El equipo visitante contraataca! ¡Con algunas jugadas afortunadas, anotan para poner el marcador 6-6!",
      "caption_ja": "アウェイチームが反撃！いくつかの幸運なプレーで得点し、6-6に！",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, night. The woodland animal Los Angeles Dodgers celebrate after a controversial play allows a run to score. The score board shows 6-6. A sly fox player bumps into the catcher causing him to drop the ball and allows a small rabbit to score. Background characters are also woodland animals. Intense emotion, varied animal expressions, dynamic action, vibrant colors. Background characters are also woodland animals."
    },
    {
      "actNumber": 3,
      "sceneNumber": 9,
      "description": "The away team takes the lead!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The away team scores again, the score is now 7-6!",
      "caption_es": "El equipo visitante anota de nuevo, ¡el marcador ahora es 7-6!",
      "caption_ja": "アウェイチームが再び得点し、スコアは7-6に！",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, night. Woodland creature, a determined wolf wearing a Los Angeles Dodgers jersey (number 3), slides safely into home plate. The scoreboard shows 7-6. The cheering crowd is composed of diverse woodland animals, filled with both excitement and disappointment. Wide shot, focus on the tension, vibrant lights, dynamic composition. Background characters are also woodland animals."
    },
    {
      "actNumber": 3,
      "sceneNumber": 10,
      "description": "Final chance for the home team! Can they pull off a win?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The home team can\'t score. The away team wins 7-6",
      "caption_es": "El equipo local no puede anotar. El equipo visitante gana 7-6",
      "caption_ja": "ホームチームは得点できず。アウェイチームが7-6で勝利",
      "imagenPrompt": "3D render, Pixar style animated kids film. Yankee Stadium, night. The Los Angeles Dodgers, represented by various woodland creatures, celebrate their victory on the field. The defeated New York Yankees woodland creature players look on in dismay. The scoreboard shows 7-6. Focus on the emotions, low angle shot, dynamic pose, vibrant lighting, wide-angle lens. Background characters are also woodland animals."
    }
  ]
}
```

### Game Story Test 9

The story output is: 
```json
{
  "story": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Ohtani steps up to the plate against Cole, but Judge catches his fly ball. Excitement fills the air as the home team prepares to bat.",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Los Angeles Dodgers face the New York Yankees! Shohei Ohtani bats first for the Dodgers, against Yankees pitcher Gerrit Cole.",
      "caption_es": "¡Los Dodgers de Los Ángeles se enfrentan a los Yankees de Nueva York! Shohei Ohtani batea primero por los Dodgers, contra el lanzador de los Yankees, Gerrit Cole.",
      "caption_ja": "ロサンゼルス・ドジャース対ニューヨーク・ヤンキース！ドジャースの大谷翔平がヤンキースの先発ピッチャー、ゲリット・コールの相手に先頭打者として登場！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The Yankees strike! Judge smashes a homer, bringing Soto home too! Chisholm Jr. follows with another homer! The home crowd erupts!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Aaron Judge hits a HUGE home run, scoring Juan Soto! Then, Jazz Chisholm Jr. hits ANOTHER home run! Yankees lead 3-0!",
      "caption_es": "¡Aaron Judge conecta un ENORME jonrón, anotando Juan Soto! ¡Luego, Jazz Chisholm Jr. conecta OTRO jonrón! ¡Los Yankees lideran 3-0!",
      "caption_ja": "アーロン・ジャッジが特大ホームランを放ち、フアン・ソトも生還！そして、ジャズ・チザムJr.がまたしてもホームラン！ヤンキースが3-0でリード！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 1,
      "sceneNumber": 3,
      "description": "The Dodgers struggle in the second inning, but the Yankees keep the pressure on. Volpe doubles, and Verdugo brings him home with a single!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Anthony Volpe hits a double! Alex Verdugo then singles, scoring Volpe! The Yankees extend their lead to 4-0.",
      "caption_es": "¡Anthony Volpe conecta un doble! ¡Alex Verdugo luego conecta un sencillo, anotando Volpe! Los Yankees extienden su ventaja a 4-0.",
      "caption_ja": "アンソニー・ボルぺがツーベースヒット！アレックス・バードゥーゴがシングルでボルぺをホームに迎え入れます！ヤンキースがリードを4-0に広げます。",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "Stanton joins the party with a solo homer in the third, making it 5-0. The Dodgers need to find a way to answer!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Giancarlo Stanton hits a solo home run! The Yankees are now leading 5-0!",
      "caption_es": "¡Giancarlo Stanton conecta un jonrón solitario! ¡Los Yankees ahora lideran 5-0!",
      "caption_ja": "ジャンカルロ・スタントンがソロホームラン！ヤンキースが5-0とリードを広げます！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The Dodgers rally! Errors and hits combine to bring five runs home, tying the game! Can they keep the momentum?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers fight back! Errors and singles score 5 runs! The game is now tied 5-5!",
      "caption_es": "¡Los Dodgers contraatacan! ¡Errores y sencillos anotan 5 carreras! ¡El juego ahora está empatado 5-5!",
      "caption_ja": "ドジャースが反撃！エラーとシングルヒットで5点を獲得！試合は5-5の同点に！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 2,
      "sceneNumber": 6,
      "description": "The Yankees answer right back, scoring one run in the sixth to retake the lead. The game is now 6-5. Tension mounts!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Yankees score a run in the 6th! They regain the lead, now 6-5!",
      "caption_es": "¡Los Yankees anotan una carrera en la 6ª! ¡Recuperan la ventaja, ahora 6-5!",
      "caption_ja": "ヤンキースが6回に1点を追加！6-5で再びリード！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 7,
      "description": "The Dodgers take the lead in the eighth, scoring two runs thanks to singles, walks, and a catcher interference! Dodgers lead 7-6!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Dodgers score 2 in the 8th! They take the lead 7-6!",
      "caption_es": "¡Los Dodgers anotan 2 en la 8ª! ¡Toman la ventaja 7-6!",
      "caption_ja": "ドジャースが8回に2点を獲得！7-6とリードを奪います！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "Judge doubles in the eighth, but the Yankees can't bring him home. The Dodgers are three outs away from victory!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Aaron Judge doubles in the 8th, but the Yankees don't score. Dodgers still lead 7-6!",
      "caption_es": "Aaron Judge conecta un doble en la 8ª, pero los Yankees no anotan. ¡Los Dodgers aún lideran 7-6!",
      "caption_ja": "アーロン・ジャッジが8回にツーベースを放つも、ヤンキースは得点できず。ドジャースが7-6でリードを維持！",
      "imagenPrompt": ""
    },
    {
      "actNumber": 3,
      "sceneNumber": 9,
      "description": "The Dodgers hold on! The Yankees go down in order in the ninth. The Dodgers win a thrilling game, 7-6!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers win! The Yankees fail to score in the bottom of the 9th. Final score: Dodgers 7, Yankees 6.",
      "caption_es": "¡Los Dodgers ganan! Los Yankees no logran anotar en la parte baja de la 9ª. Resultado final: Dodgers 7, Yankees 6.",
      "caption_ja": "ドジャースが勝利！ヤンキースは9回裏に得点できず。最終スコア：ドジャース7、ヤンキース6。",
      "imagenPrompt": ""
    }
  ]
}
```

The storyboard_prompts output is:
```json
{
  "story": [
    {
      "actNumber": 1,
      "sceneNumber": 1,
      "description": "The game begins! Ohtani steps up to the plate against Cole, but Judge catches his fly ball. Excitement fills the air as the home team prepares to bat.",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Los Angeles Dodgers face the New York Yankees! Shohei Ohtani bats first for the Dodgers, against Yankees pitcher Gerrit Cole.",
      "caption_es": "¡Los Dodgers de Los Ángeles se enfrentan a los Yankees de Nueva York! Shohei Ohtani batea primero por los Dodgers, contra el lanzador de los Yankees, Gerrit Cole.",
      "caption_ja": "ロサンゼルス・ドジャース対ニューヨーク・ヤンキース！ドジャースの大谷翔平がヤンキースの先発ピッチャー、ゲリット・コールの相手に先頭打者として登場！",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: Shohei Ohtani, a determined badger wearing a Los Angeles Dodgers uniform (#17), batting against Gerrit Cole, a grumpy bear in a New York Yankees uniform (#45). Aaron Judge, a majestic lion wearing a Yankees uniform (#99), catches the fly ball. The stadium is packed with woodland creatures cheering wildly. Vibrant colors, dynamic action shot. Focus on the batter and catcher."
    },
    {
      "actNumber": 1,
      "sceneNumber": 2,
      "description": "The Yankees strike! Judge smashes a homer, bringing Soto home too! Chisholm Jr. follows with another homer! The home crowd erupts!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Aaron Judge hits a HUGE home run, scoring Juan Soto! Then, Jazz Chisholm Jr. hits ANOTHER home run! Yankees lead 3-0!",
      "caption_es": "¡Aaron Judge conecta un ENORME jonrón, anotando Juan Soto! ¡Luego, Jazz Chisholm Jr. conecta OTRO jonrón! ¡Los Yankees lideran 3-0!",
      "caption_ja": "アーロン・ジャッジが特大ホームランを放ち、フアン・ソトも生還！そして、ジャズ・チザムJr.がまたしてもホームラン！ヤンキースが3-0でリード！",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: Aaron Judge, the majestic lion wearing a New York Yankees uniform (#99), hitting a massive home run. Juan Soto, a speedy rabbit also in a Yankees uniform (#22), is rounding the bases, high-fiving Jazz Chisholm Jr., a playful monkey also in a Yankees uniform (#2). The stadium is exploding with celebration. Focus on the Judge and the excitement of the crowd."
    },
    {
      "actNumber": 1,
      "sceneNumber": 3,
      "description": "The Dodgers struggle in the second inning, but the Yankees keep the pressure on. Volpe doubles, and Verdugo brings him home with a single!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Anthony Volpe hits a double! Alex Verdugo then singles, scoring Volpe! The Yankees extend their lead to 4-0.",
      "caption_es": "¡Anthony Volpe conecta un doble! ¡Alex Verdugo luego conecta un sencillo, anotando Volpe! Los Yankees extienden su ventaja a 4-0.",
      "caption_ja": "アンソニー・ボルぺがツーベースヒット！アレックス・バードゥーゴがシングルでボルぺをホームに迎え入れます！ヤンキースがリードを4-0に広げます。",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: Anthony Volpe, a sly fox wearing a New York Yankees uniform (#11), sliding into second base with a double. Alex Verdugo, a strong bull also in a Yankees uniform (#24), is hitting a single, sending Volpe home. Dodger players, represented by sad-looking armadillos, are dejected in the background. Yankee stadium filled with cheering woodland creatures. Focus on the action at home plate."
    },
    {
      "actNumber": 2,
      "sceneNumber": 4,
      "description": "Stanton joins the party with a solo homer in the third, making it 5-0. The Dodgers need to find a way to answer!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Giancarlo Stanton hits a solo home run! The Yankees are now leading 5-0!",
      "caption_es": "¡Giancarlo Stanton conecta un jonrón solitario! ¡Los Yankees ahora lideran 5-0!",
      "caption_ja": "ジャンカルロ・スタントンがソロホームラン！ヤンキースが5-0とリードを広げます！",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: Giancarlo Stanton, a powerful gorilla wearing a New York Yankees uniform (#27), rounding the bases after hitting a solo home run. The woodland creature fans in Yankee Stadium are going wild. On the pitcher's mound, the Los Angeles Dodgers pitcher, a bewildered penguin, looks defeated. Focus on the celebration and the dejected pitcher."
    },
    {
      "actNumber": 2,
      "sceneNumber": 5,
      "description": "The Dodgers rally! Errors and hits combine to bring five runs home, tying the game! Can they keep the momentum?",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers fight back! Errors and singles score 5 runs! The game is now tied 5-5!",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: A chaotic scene with Los Angeles Dodgers players, now energized badgers, (including Shohei Ohtani, the same badger from scene 1), celebrating as they score runs. A panicked squirrel representing a Yankees fielder throws the ball erratically. Woodland creature fans in Dodger Stadium erupt in cheers. Scoreboard shows 5-5. Focus on the exciting chaos and joyful Dodger players."
    },
    {
      "actNumber": 2,
      "sceneNumber": 6,
      "description": "The Yankees answer right back, scoring one run in the sixth to retake the lead. The game is now 6-5. Tension mounts!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Yankees score a run in the 6th! They regain the lead, now 6-5!",
      "caption_es": "¡Los Yankees anotan una carrera en la 6ª! ¡Recuperan la ventaja, ahora 6-5!",
      "caption_ja": "ヤンキースが6回に1点を追加！6-5で再びリード！",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: The Yankees players, including the majestic lion Aaron Judge (#99), are celebrating a run scored. The umpire, a wise owl, signals safe. The Los Angeles Dodgers players, now determined badgers, look on with concern. Scoreboard shows 6-5. Focus on the victorious Yankee team and the tension in the air."
    },
    {
      "actNumber": 3,
      "sceneNumber": 7,
      "description": "The Dodgers take the lead in the eighth, scoring two runs thanks to singles, walks, and a catcher interference! Dodgers lead 7-6!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Dodgers score 2 in the 8th! They take the lead 7-6!",
      "caption_es": "¡Los Dodgers anotan 2 en la 8ª! ¡Toman la ventaja 7-6!",
      "caption_ja": "ドジャースが8回に2点を獲得！7-6とリードを奪います！",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: Two Los Angeles Dodgers players, represented by energetic badgers, scoring runs in the 8th. One is sliding into home plate, with the Yankees catcher, a flustered raccoon, arguing with the umpire. The scoreboard in Dodger Stadium displays 7-6, Dodgers in the lead. Cheering woodland creatures fill the stands. Focus on the action at home plate and the catcher's frustration."
    },
    {
      "actNumber": 3,
      "sceneNumber": 8,
      "description": "Judge doubles in the eighth, but the Yankees can't bring him home. The Dodgers are three outs away from victory!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "Aaron Judge doubles in the 8th, but the Yankees don't score. Dodgers still lead 7-6!",
      "caption_es": "Aaron Judge conecta un doble en la 8ª, pero los Yankees no anotan. ¡Los Dodgers aún lideran 7-6!",
      "caption_ja": "アーロン・ジャッジが8回にツーベースを放つも、ヤンキースは得点できず。ドジャースが7-6でリードを維持！",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: Aaron Judge, the majestic lion wearing his Yankees uniform (#99), stands on second base after hitting a double. The Los Angeles Dodgers players, alert badgers, are positioned defensively. The tension in the air is palpable. Scoreboard shows 7-6, Dodgers in the lead. Focus on Judge on second and the worried looks of the Dodger team."
    },
    {
      "actNumber": 3,
      "sceneNumber": 9,
      "description": "The Dodgers hold on! The Yankees go down in order in the ninth. The Dodgers win a thrilling game, 7-6!",
      "imageUrl": "",
      "audioUrl_en": "",
      "audioUrl_es": "",
      "audioUrl_ja": "",
      "caption_en": "The Dodgers win! The Yankees fail to score in the bottom of the 9th. Final score: Dodgers 7, Yankees 6.",
      "caption_es": "¡Los Dodgers ganan! Los Yankees no logran anotar en la parte baja de la 9ª. Resultado final: Dodgers 7, Yankees 6.",
      "caption_ja": "ドジャースが勝利！ヤンキースは9回裏に得点できず。最終スコア：ドジャース7、ヤンキース6。",
      "imagenPrompt": "3D rendered Pixar style, animated kids film: The Los Angeles Dodgers players, jubilant badgers, celebrate their victory on the field. Shohei Ohtani, the standout badger player, is hoisted onto the shoulders of his teammates. The Yankees players, including the disappointed lion Aaron Judge, walk off the field dejectedly. The scoreboard shows the final score: Dodgers 7, Yankees 6. The crowd of woodland creatures cheers wildly. Focus on the victorious Dodgers team."
    }
  ]
}
```

### Game Story Test 8

```json
{
  "title": "The Great Baseball Battle",
  "acts": [
    {
      "actNumber": 1,
      "scenes": [
        {
          "sceneNumber": 1,
          "description": "The game begins with a series of outs, followed by a big hit that brings the first score for the home team!",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
          "caption": "Shohei Ohtani steps up to bat but flies out!  Mookie Betts and Freddie Freeman also get out, and the away team's turn is done. Then, the home team's Gleyber Torres grounds out but Juan Soto gets on base with a walk!  Suddenly, Aaron Judge hits a HUGE home run and brings Juan Soto home too, making it 2-0!",
          "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene of a sunny baseball field with bright green grass and a clear blue sky.  Shohei Ohtani, a bear wearing a light blue Los Angeles Dodgers uniform with jersey number 17, flies out at bat. Mookie Betts, a speedy squirrel in a light blue Los Angeles Dodgers uniform with jersey number 50, and Freddie Freeman, a relaxed dog in a light blue Los Angeles Dodgers uniform with jersey number 5, are looking dejected. Gleyber Torres, a quick fox in a pinstriped New York Yankees uniform with jersey number 25, grounds out at bat. Juan Soto, a sly cat in a pinstriped New York Yankees uniform with jersey number 22, is on first base after a walk. Aaron Judge, a majestic lion wearing a pinstriped New York Yankees uniform with jersey number 99, is swinging his bat with incredible force, connecting with the ball and sending it flying high. Juan Soto, the sly cat with jersey number 22, is rounding the bases, a huge smile on his face. The woodland animal crowd in the stands is cheering wildly. The Los Angeles Dodgers players, the bear, the squirrel, and the dog, in their light blue uniforms, are on the field looking surprised. The action is vibrant and dynamic, highlighting the exciting movement and big hit."
        },
        {
          "sceneNumber": 2,
          "description": "The home team keeps the momentum with another home run, adding to their lead, but the inning ends quickly.",
           "imageUrl": "TBD",
          "audioUrl": "TBD",
           "caption": "It doesn't stop there, Jazz Chisholm Jr. hits another home run, making the score 3-0! Giancarlo Stanton lines out and Anthony Rizzo strikes out swinging, so the home team's turn is over. The away team comes up to bat, Teoscar Hernández and Max Muncy strike out. Then, Enrique Hernández flies out to end the inning!",
          "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene of a bright baseball field with cheering woodland animal fans in the stands. Jazz Chisholm Jr., a swift cheetah in a pinstriped New York Yankees uniform with jersey number 2, is mid-swing, the ball rocketing off his bat, with sparks around the bat. The ball is going up and away, with the first base coach, a supportive badger in a New York Yankees uniform, cheering him on. The outfielders of the Los Angeles Dodgers, Teoscar Hernández (a hawk in a light blue uniform with jersey number 37) and Max Muncy (a lumbering ox in a light blue uniform with jersey number 13) are looking up at the ball going over their heads. Giancarlo Stanton, a powerful gorilla in pinstripes with jersey number 27, lines out, and Anthony Rizzo, a friendly bear in pinstripes with jersey number 48, strikes out swinging. Teoscar Hernández and Max Muncy strike out for the Dodgers, and Enrique Hernández (a nimble rabbit in a light blue uniform with jersey number 8) flies out to end the inning. The scene is dynamic and colorful, showcasing the excitement of the home run."
        },
        {
          "sceneNumber": 3,
          "description": "The home team manages to score again as the action goes back and forth, but the away team remains scoreless.",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
         "caption": "Anthony Volpe hits a double, and then scores on a single by Alex Verdugo which makes it 4-0! The away team comes to bat, Tommy Edman pops out, Will Smith pops out, Gavin Lux walks and then Shohei Ohtani flies out.  Then, Giancarlo Stanton hits another home run, making it 5-0!  The away team is struggling, and it’s still all home team!",
          "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene of a bright baseball field. Anthony Volpe, a quick mouse in a pinstriped New York Yankees uniform with jersey number 11, is sliding into second base on a line drive, with Teoscar Hernandez, the hawk from the Los Angeles Dodgers in a light blue uniform with jersey number 37, trying to get the ball. The outfield is bright, and the stadium is in the background. The New York Yankees are cheering him on from the dugout. The Los Angeles Dodgers players, Tommy Edman (a fox in a light blue uniform with jersey number 19) and Will Smith (a turtle in a light blue uniform with jersey number 16), pop out. Gavin Lux, (a sleek wolf in a light blue uniform with jersey number 2), walks and Shohei Ohtani flies out again. Giancarlo Stanton, the gorilla with jersey number 27, hits another home run, making the score 5-0! The scene showcases the dynamic base-running and continued home team dominance, all in a vibrant and engaging style."
        }
      ]
    },
    {
      "actNumber": 2,
      "scenes": [
        {
          "sceneNumber": 4,
           "description": "The away team tries to rally, but the home team's defense holds, before the away team finally strikes back!",
           "imageUrl": "TBD",
          "audioUrl": "TBD",
           "caption": "The away team's Mookie Betts walks, Freddie Freeman flies out and Teoscar Hernández grounds out before Max Muncy lines out to end the inning.  Alex Verdugo walks, Juan Soto hits a single, and it looks like the home team will score again, but Aaron Judge and Jazz Chisholm Jr.  get out! But, in the next inning the away team scores 5 runs to tie the game!  It's all tied up!",
          "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene on a baseball field. Mookie Betts, the squirrel from the Los Angeles Dodgers in a light blue uniform with jersey number 50, is walking onto first base. The home team pitcher, a stoic bison in a pinstriped New York Yankees uniform, is on the mound looking down, his brow furrowed. Freddie Freeman, the dog in the Dodgers uniform with jersey number 5, flies out, Teoscar Hernández, the hawk with jersey number 37, grounds out, and Max Muncy, the ox with jersey number 13, lines out, ending the inning. The scene transitions to Alex Verdugo, a determined bull in a pinstriped New York Yankees uniform with jersey number 24, walking to first, and Juan Soto, the cat with jersey number 22, hitting a single. Aaron Judge, the lion with jersey number 99, and Jazz Chisholm Jr., the cheetah with jersey number 2, both get out. The scene shifts to show the away team, Los Angeles Dodgers, celebrating their 5-run tie, all looking excited and determined. The animation is dynamic, capturing the tension and excitement of the back and forth action of the game."
        },
          {
          "sceneNumber": 5,
          "description": "Both teams exchange blows, the away team gets their first lead but the home team claws back!",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
           "caption": "Enrique Hernández and Tommy Edman manage singles, then with some errors and smart plays the away team score 5 runs! Then the home team bats and gets runners on base but fail to score.  The next inning the home team manages to score one run by using a sacrifice fly, bringing the score to 6-5. The home team is back in the lead! The drama is at its highest!",
          "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene of a bustling baseball diamond, with runners on bases. Enrique Hernández, the rabbit with jersey number 8, and Tommy Edman, the fox with jersey number 19, both in Los Angeles Dodgers light blue uniforms, manage singles and with some errors and smart plays the Los Angeles Dodgers score 5 runs! The excited Dodgers players, the squirrel, dog, ox, hawk, rabbit and fox, are cheering each other. The home team New York Yankees, the lion, cat, cheetah, mouse, gorilla and bull in their pinstripes, look on slightly dejected but ready to fight back. The home team bats and gets runners on base, but fails to score. In the next inning, a New York Yankees player scores one run using a sacrifice fly, bringing the score to 6-5. The scene shows the back-and-forth momentum, captured with dynamic movement and expressions of both joy and determination."
        },
         {
          "sceneNumber": 6,
          "description": "The away team has a late rally, with some help of catcher interference, they take the lead, but the home team is still in it!",
          "imageUrl": "TBD",
         "audioUrl": "TBD",
          "caption": "The away team keeps pushing and gets some walks and hits! Then with a bit of catcher interference, they tie it up, and then they manage to get more runners in to make it 7-6!  The away team is in the lead for the first time!  Can they hang on?  The home team's Juan Soto gets out, then Aaron Judge gets a double and then another walk but they can't bring anyone in. It's nail-biting baseball!",
         "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene focused on the baseball field. Austin Wells, a frustrated beaver in a pinstriped New York Yankees uniform, with jersey number 20, the catcher, is looking down in frustration. There are runners on first and second, and a Los Angeles Dodger batter, the determined dog in a light blue uniform with jersey number 5, is standing on first due to catcher interference. The Los Angeles Dodgers players, the squirrel, the hawk, and the ox, in their light blue uniforms, are looking on, some grinning with anticipation. The Los Angeles Dodgers are pushing hard, getting walks and hits, and tie the game, then take the lead 7-6. The New York Yankees, Juan Soto the cat with jersey number 22, gets out, then Aaron Judge, the lion with jersey number 99, gets a double, followed by a walk. The scene is filled with tension, with the detailed facial expressions and the exciting action."
        }
      ]
    },
    {
      "actNumber": 3,
       "scenes": [
        {
          "sceneNumber": 7,
          "description": "The away team tries to add more runs, but the home team's pitching holds them off.",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
         "caption": "The away team gets a couple of singles and walks, but don't manage to get any more runs. The home team’s hitters are looking to tie or take the lead.  It's the last chance for the home team to take it!",
         "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene of a baseball field focused on the pitcher. The home team pitcher, a determined bison in a pinstriped New York Yankees uniform, is throwing a fast ball. The Los Angeles Dodgers batter, a tensed hawk in a light blue uniform with jersey number 37, stands ready but slightly nervous. There are runners on base, but the home team infielders, the mouse, the bull, the lion and the cat, are ready to field the next play. The scene captures the intensity and tension, highlighting the crucial moment in the game."
        },
        {
          "sceneNumber": 8,
          "description": "The home team comes to bat but is unable to score and the away team wins the game!",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
          "caption": "It’s the bottom of the 9th, and the home team needs to score!  But, Anthony Volpe grounds out, and then Austin Wells and Alex Verdugo both strike out. The game is over! The away team has won a dramatic battle!",
           "imagenPrompt": "A 3D rendered, Pixar-style animated kids film scene of a baseball field at twilight. The focus is on the home team player, Alex Verdugo, the bull in a pinstriped New York Yankees uniform with jersey number 24, who just struck out and is walking back to the dugout looking sad. Anthony Volpe, the mouse in pinstripes with jersey number 11, grounds out, and Austin Wells, the beaver in pinstripes with jersey number 20, also strikes out. The away team Los Angeles Dodgers, the squirrel, dog, ox, hawk, rabbit, and fox, are in the background cheering and hugging each other. It is a scene of the victorious Los Angeles Dodgers, while the home team is dejected. The lighting is soft, emphasizing the emotion of the game's conclusion."
        }
      ]
    }
  ]
}
```

### Game Story Test 7

```json
{
  "title": "The Great Baseball Battle",
  "acts": [
    {
      "actNumber": 1,
      "title": "The First Innings",
      "scenes": [
        {
          "sceneNumber": 1,
          "description": "The game begins with both teams trying to get the ball in play, but strong pitching and fielding keep them at bay.",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
          "caption": "The game started with Ohtani hitting a fly ball, followed by Betts and Freeman doing the same for the top half of the 1st. For the home team, Torres grounded out, Soto walked, and then Judge hit a HUGE home run, bringing Soto home! But wait, Chisholm Jr. hits ANOTHER homer right after, and the home team is up 3-0! The first inning ends with Stanton, Rizzo, and a strikeout to end the bottom of the inning.",
             "imagenPrompt": "A sunny baseball stadium. The away team (wearing light-colored uniforms) is batting against the home team (wearing dark-colored uniforms). Ohtani is hitting a fly ball to center field, with Judge making a catch in the distance. The home team celebrates after Judge\'s home run, and Chisholm Jr. is rounding the bases after his own home run."
        },
        {
          "sceneNumber": 2,
          "description": "The second inning sees more batters trying to get on base, with a double and a single getting a run, but the defenses of both teams keep the score close.",
            "imageUrl": "TBD",
            "audioUrl": "TBD",
            "caption": "The second inning starts with three strikeouts against the away team. For the home team, Volpe hit a double, followed by a ground out where he made it to 3rd. Then Verdugo hit a single, bringing Volpe home! More walks for the home team. The inning ends on a groundout.",
            "imagenPrompt": "A baseball field, Volpe running to 2nd after his double while the away team looks on. Verdugo is hitting the single that brings in Volpe."
        },
         {
          "sceneNumber": 3,
          "description": "More bats in the third inning, with another home run added to the home team\'s score.",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
          "caption": "Three outs in the top of the third for the away team. Then, for the home team, Stanton hits a home run adding another run! The third inning ends with Rizzo, Volpe, and Wells unable to score.",
          "imagenPrompt": "A baseball field, Giancarlo Stanton running around the bases after hitting a home run with a bright blue sky in the background."
        }
      ]
    },
    {
      "actNumber": 2,
      "title": "The Turning Tides",
       "scenes": [
          {
            "sceneNumber": 4,
            "description": "Inning 4 saw more strong pitching and a few more hits but no runs until the bottom half of the inning, which saw a few singles.",
             "imageUrl": "TBD",
             "audioUrl": "TBD",
            "caption": "The fourth inning starts with the away team with 3 outs and no runs. Then the home team got walks and singles, with some players making it to second and third base, but no runs scored this inning.",
             "imagenPrompt":"A baseball field, Mookie Betts catching a fly ball in right field, and other players standing in the background. Then, on the bottom half, Juan Soto hits a single while the away team watches."
         },
         {
           "sceneNumber": 5,
            "description": "The away team begins a big comeback in the fifth, getting base hits and runs.",
             "imageUrl": "TBD",
            "audioUrl": "TBD",
           "caption": "In the top of the fifth, the away team\'s offense began to come alive! A series of singles, doubles, and a couple of errors brought them 5 runs and tied the game! The bottom half saw a few base hits for the home team, but no runs were scored.",
           "imagenPrompt": "A baseball field, The away team is celebrating after scoring multiple runs, running to and from the bases. The home team is looking dejected in the background."
         },
         {
            "sceneNumber": 6,
            "description": "The home team regained the lead in the sixth inning with strategic walks and a sacrifice fly.",
            "imageUrl": "TBD",
            "audioUrl": "TBD",
            "caption": "In the top of the sixth, the away team went out quickly, but the home team rallied back! Several walks, a force out, and a sacrifice fly brought a run home and the home team was up by 1!",
            "imagenPrompt": "A baseball field, home team players cheering from the dugout as their teammates run to the bases. A player, Stanton, is hitting a sacrifice fly."
          }
        ]
    },
    {
      "actNumber": 3,
      "title": "The Final Innings",
      "scenes": [
        {
          "sceneNumber": 7,
          "description": "Both teams battled with many walks and hits, but the defenses held until the 8th.",
          "imageUrl": "TBD",
          "audioUrl": "TBD",
            "caption":"The top of the seventh saw many outs for the away team and a couple of walks. The bottom saw the same with strikeouts and groundouts. In the top of the 8th, the away team rallied again! Several singles and walks, plus errors, led to two more runs! The bottom of the 8th saw another double from Judge, but no runs scored.",
           "imagenPrompt": "A baseball field, batters hitting and running, players catching, and pitchers pitching all around the field. Judge hitting a sharp double to left field."
        },
        {
          "sceneNumber": 8,
          "description": "The away team tried to add to their lead, but the home team defense stopped them.",
            "imageUrl": "TBD",
            "audioUrl": "TBD",
            "caption": "The top of the ninth saw the away team get some hits, but none were enough to get any runs. The game ended with the home team not getting a run.",
            "imagenPrompt": "A baseball field, a player from the away team singles, a player from the home team catches a fly ball. A player from the home team strikes out."
          },
          {
            "sceneNumber": 9,
             "description": "The game concludes with the away team winning, leaving the home team sad but ready to fight another day.",
              "imageUrl": "TBD",
              "audioUrl": "TBD",
              "caption": "The game ended with the away team winning. They came back from behind to win by one run. But it was a close game till the very end!",
              "imagenPrompt": "A baseball field, the away team celebrating their win. The home team looking disappointed but still giving each other high fives."
        }
      ]
    }
  ]
}
```

Okay, here are the key visual scenes, along with their corresponding AI prompts featuring animal characters in a Pixar-style animated film look.

**Scene Description and AI Prompt**

**Scene 1:**

*   **Description:** The game begins with both teams trying to get the ball in play, but strong pitching and fielding keep them at bay. Ohtani hits a fly ball to center field. Judge makes a catch. Home team celebrates Judge\'s home run. Chisholm Jr. is rounding the bases after his own home run.

*   **AI Prompt:** "A wide shot of a baseball stadium filled with a lively crowd of woodland creatures. The away team, the Los Angeles Dodgers, are represented by a team of light-colored foxes, all wearing their iconic blue and white uniforms with a stylized "LA" logo on their caps. On the plate, a fox with jersey #17 (Ohtani) hits a fly ball to center field. In the field, the home team, the New York Yankees, are represented by a team of dark brown bears wearing pinstripe uniforms with the NY logo. A bear with jersey #99 (Judge) makes the catch in center field. The Yankees dugout, filled with cheering bears, celebrates the home run. The camera shifts focus to show a small and swift cheetah with jersey #3 (Chisholm Jr.) rounding the bases after hitting his own homer. The overall style is 3D rendered, Pixar-style animated kids film, emphasizing bright colors, dynamic action, and a sense of playful competition. The weather is sunny and clear."

**Scene 2:**

*   **Description:** The second inning starts with three strikeouts against the away team. Volpe hits a double, followed by a ground out where he makes it to 3rd. Then Verdugo hits a single, bringing Volpe home.

*   **AI Prompt:** "A medium shot of the baseball field, showcasing a fox with #17 (Ohtani), returning to the dugout after striking out. Next the focus shifts to a beaver with jersey #11 (Volpe) running hard to second base, leaving a dust cloud behind him after a double. A shot of a marmot with #28 (Verdugo), connecting with a single as Volpe slides into home. The other Dodger foxes look on as the Yankees bears cheer. The environment should be a 3D rendered, Pixar-style animated kids film look, emphasizing movement and excitement, set against a partly cloudy sky."

**Scene 3:**

*   **Description:** More bats in the third inning, with another home run added to the home team\'s score. Stanton hits a home run.

*   **AI Prompt:** "A dynamic close-up shot of a massive rhinoceros with jersey #27 (Stanton), swinging his bat and connecting with a pitch. The ball is shown flying through the air, trailing a streak. The camera then follows the rhino running triumphantly around the bases, set against a bright blue sky, with the stadium full of cheering woodland animals. Use a 3D rendered, Pixar-style animated kids film style for vibrant colors and dramatic action."

**Scene 4:**

*   **Description:** Inning 4 saw more strong pitching and a few more hits but no runs until the bottom half of the inning, which saw a few singles. Mookie Betts catches a fly ball in right field. Juan Soto hits a single

*   **AI Prompt:** "A wide shot of a fox with jersey #50 (Betts) catching a fly ball with a determined expression in right field, with other Dodger foxes and Yankee bears looking on in the background. Then, the focus switches to a polar bear with jersey #22 (Soto) connecting with the ball and running towards first, in a Pixar-style 3D rendering. The stadium is filled with cheering and engaging woodland animal crowd."

**Scene 5:**

*   **Description:** The away team begins a big comeback in the fifth, getting base hits and runs.

*  **AI Prompt:** "A dynamic action shot of several Dodger foxes running the bases, celebrating after scoring multiple runs. The fox players are jumping, high-fiving, and showing excitement. In the background, the Yankee bears appear dejected, with slumped postures. The style is 3D rendered, Pixar-style animated kids film, with vibrant colors and a strong emphasis on the movement and emotions of the characters, all set against a vibrant baseball field filled with trees in the background."

**Scene 6:**

*  **Description:** The home team regained the lead in the sixth inning with strategic walks and a sacrifice fly. Stanton hits a sacrifice fly.

*   **AI Prompt:** "A medium shot of the Yankee bear dugout, filled with cheering bears. The focus is on the rhino with #27 (Stanton), hitting a sacrifice fly. The style is a 3D rendered, Pixar-style animated kids film, with a focus on the expressions of the characters and the dynamic action of the sacrifice fly. The baseball stadium is lively, filled with woodland creatures, and lit with a warm afternoon glow."

**Scene 7:**

*   **Description:** Both teams battled with many walks and hits, but the defenses held until the 8th. Judge hits a sharp double to left field.

*   **AI Prompt:** "A wide, action-packed shot showcasing the baseball field where foxes and bears are running, hitting, and catching the ball. The focus shifts to Judge hitting a sharp double to left field, with the ball clearly shown in motion. The style should be a 3D rendered, Pixar-style animated kids film look, with emphasis on the energy of the play and the varied actions of the animal characters on the field."

**Scene 8:**

*  **Description:** The away team tried to add to their lead, but the home team defense stopped them. A player from the away team singles, a player from the home team catches a fly ball, a player from the home team strikes out.

*   **AI Prompt:** "A three-part sequence: First, a fox from the Dodgers singles. Second, a bear from the Yankees catches a fly ball in center field. Third, a bear from the Yankees strikes out, with the action displayed across the frame. The animation style is 3D rendered, Pixar-style, with emphasis on the fast-paced action and varied poses of the animals. The background features woodland animals in the stands and a partly cloudy sky."

**Scene 9:**

*   **Description:** The game concludes with the away team winning, leaving the home team sad but ready to fight another day.

*  **AI Prompt:** "A wide, celebratory shot of the Los Angeles Dodger foxes cheering and jumping in the air after winning. On the other side, the New York Yankee bears are looking dejected but still giving each other high fives with a determined look in their eyes. The stadium is filled with cheering woodland animals, The overall scene is a 3D rendered, Pixar-style animation, with vibrant colors and a sense of closure."

### Game Story Test 6

**Act 1: The Beginning of the Battle**

Scene 1: The game started with a mighty swing by Shohei Ohtani, but it flew right into Aaron Judge's glove in center field! Mookie Betts tried next but grounded out to first base. Then, Freddie Freeman popped one up to left field, and the first inning was off to a quiet start for the visiting team.

Scene Break

Scene 2: Now it was the home team's turn! Gleyber Torres hit a grounder, but it was scooped up by Gavin Lux. Then, Juan Soto walked to first base. The crowd roared as Aaron Judge stepped to the plate and CRACK! He hit a HUGE home run, scoring himself and Juan Soto! The home team was up 2-0.

Scene Break

Scene 3: The home team wasn't done yet! Next up, Jazz Chisholm Jr. hit ANOTHER home run, making the score 3-0! It was a big, exciting start for the home team. Giancarlo Stanton hit a line drive that was caught, and Anthony Rizzo struck out to end the explosive first inning.

Scene Break

**Act 2: The Seesaw of Action**

Scene 4: The visiting team was determined to catch up. In the second inning, there were strikeouts and a flyout.  Then, in the bottom of the second, Anthony Volpe hit a double and eventually scored on a hit by Alex Verdugo. The score went up to 4-0! It seemed like the home team was unstoppable with a few walks to load the bases, but a ground out stopped the rally!

Scene Break

Scene 5: In the third, the home team added another run with a HUGE home run by Giancarlo Stanton, now 5-0. But wait! In the 5th inning, the visiting team rallied! It started with a hit by Enrique Hernandez and followed by a few errors and a single from Mookie Betts. Then, Freddie Freeman hit a single, scoring two runners! Then Teoscar Hernández cracked a double, scoring two more! Suddenly the score was tied at 5-5!  The game was tied and the excitement was building.

Scene Break

Scene 6: Both teams were fighting hard. In the bottom of the 5th the home team loaded the bases, but could not score! The home team took the lead again in the sixth, when Giancarlo Stanton hit a sacrifice fly, bringing Juan Soto home making the score 6-5. The lead went back to the home team! Both teams had players on base but couldn’t get a run in the 7th inning.

Scene Break

**Act 3: The Final Showdown**

Scene 7: The top of the eighth inning started with more hits and walks, bringing in another run for the visiting team. A sacrifice fly by Mookie Betts brought another run making the score 7-6 in favor of the visiting team! The visitors had taken the lead for the first time!

Scene Break

Scene 8: In the bottom of the 8th, Aaron Judge hit a double, and Jazz Chisholm Jr walked, giving the home team a chance but they couldn't score, and now it was the top of the ninth! Could the home team pull off a miracle?

Scene Break

Scene 9: In the top of the 9th the visiting team got another runner on from a single by Teoscar Hernández, followed by a walk to Enrique Hernández! But it was no good! The next three batters did not score.

Scene Break

Scene 10: The home team was down to their last three outs. Anthony Volpe, Austin Wells and Alex Verdugo all got out! The visiting team won the game by the skin of their teeth 7-6. It was a wild back-and-forth game that kept everyone on the edge of their seats until the very last out!

Okay, here's a breakdown of the key visual scenes for Act 1, followed by the AI prompts, focusing on animal characters in a Pixar-style animated film.

**Key Visual Scenes for Act 1 (Scenes 1-3)**

*   **Scene 1:** Ohtani's powerful swing is caught by Judge in center field. Close-up on Ohtani’s bat making contact.
*   **Scene 2:** Judge hits a massive home run. Wide shot of the ball soaring with Judge rounding first.
*   **Scene 3:** Chisholm hits another home run, adding to the home team's lead. Medium shot from side view showing the swing and the ball launching.

**Scene 1**

**Scene Description:** A powerful swing from a Los Angeles Dodgers batter. The ball is struck with force and is high in the air traveling to center field. A New York Yankees fielder catches the ball in his glove. The scene highlights the tension and athleticism of the opening play.

**AI Prompt:**
"""
A dynamic 3D rendered Pixar-style animation scene showing a lion wearing a Los Angeles Dodgers uniform (number 17 on his back), intensely swinging a bat. The baseball is a blur, flying towards the center of the field. A large, powerful grizzly bear, wearing a New York Yankees uniform (number 99 on his back), makes a leaping catch in center field, glove extended. The backdrop shows a packed baseball stadium, filled with various woodland creatures, such as squirrels, rabbits, and chipmunks as spectators, dressed in team colors and creating a vibrant atmosphere. The overall scene is action-packed and brightly lit, emphasizing the start of the game.
"""

**Scene 2**

**Scene Description:** The camera focuses on the batter at the plate. He makes a big swing and the ball is launched through the air. The home team cheers as he rounds the bases.

**AI Prompt:**
"""
A wide-angle, 3D rendered Pixar-style animated scene depicting a roaring grizzly bear, wearing a New York Yankees uniform (number 99 on his back), connecting with a baseball, sending it soaring out of the stadium for a home run.  He rounds first base with a determined look, his body in mid-stride. A happy crowd of woodland creatures in the stands jumps and cheers. The dugout behind the bear shows a group of excited New York Yankees team members, also represented by various animals, celebrating the home run. The stadium is brightly lit with the sun setting in the background, creating a vibrant and exciting atmosphere.
"""

**Scene 3**

**Scene Description:** A panther hits a powerful home run for the home team. The scene conveys power and excitement as the home team builds an early lead.

**AI Prompt:**
"""
A dynamic, medium shot, 3D rendered Pixar-style animated scene showing a sleek panther, wearing a New York Yankees uniform (number 2 on his back), swinging the bat with a follow-through motion.  The baseball is traveling rapidly, depicted as a blur. The shot emphasizes the power of the swing and the home run.  Behind the panther, in the stands, various woodland creatures such as foxes, owls and badgers are seen cheering enthusiastically. The stadium is filled with vibrant lights and colors, highlighting the electrifying atmosphere of the game.
"""


### Game Story Test 4

'Okay, here's a breakdown of key scenes, along with corresponding advanced AI prompts using animal characters in a Pixar-style animated setting:

**Key Visual Scenes (5-15):**

1.  **Quiet Top of the First:** Empty field with scattered woodland creatures in the stands.
2.  **Judge's Homer:** A roaring lion (Aaron Judge) hitting a ball out of the park with a bat, bats swinging.
3.  **Chisholm Jr.'s Homer:** A leaping cheetah (Jazz Chisholm Jr.) connecting with a home run and flying through the air.
4.  **Second Inning Single:** A leaping fox (Yankee player) runs from second to third.
5.  **Stanton's Solo Homer:** A hulking bear (Giancarlo Stanton) hitting a deep home run, a flying ball.
6.  **Away Team Strikeout:** A frustrated coyote (Dodger player) swinging and missing a pitch.
7.  **Away Team Single & Scoring Run:** A fast rabbit (Dodger player) sprints to home and scores.
8.  **Another Away Team Single & Scoring:** Two racoons (Dodgers players) on base are brought home.
9.  **Tying Double:** A determined skunk (Dodger player) hitting a double.
10. **Home Team Sacrifice Fly:** A bat (Yankee player) flies out to the outfield to bring a runner in.
11. **Away Team Sacrifice Fly Tie:** A swift hawk (Dodger player) flies out bringing a runner in.
12. **Away Team Lead Sacrifice Fly:** Another hawk (Dodger player) flies out bringing in a runner to take the lead.
13. **Judge's Double:** The lion (Aaron Judge) hits a line drive down the line.
14. **Final Out:** A somber raccoon (Yankee player) strikes out.
15. **Away Team Celebration:** A group of joyful and victorious Dodgers, running around and leaping in celebration.

---

**AI Prompts:**

**Scene 1: Quiet Top of the First**

*   **Description:** Empty baseball field, bright, sunny sky. Woodland animals are seen in the stands - squirrels, rabbits, and owls. The New York Yankee (home) dugout is to the left and is occupied by a single bored bear. The Los Angeles Dodgers (away) dugout on the right is empty.
*   **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A wide shot of an empty baseball field with vibrant green grass under a partly cloudy sky. Small woodland animals – squirrels, rabbits, and owls – sit scattered in the stands. The New York Yankees dugout (left) is shown with a single bored bear wearing a New York Yankees jersey with no number.  The Los Angeles Dodgers dugout (right) is completely empty. The scene should have a bright, cheerful tone despite the lack of action, set at Yankee Stadium on a sunny afternoon, 2024-10-30."

**Scene 2: Judge's Homer**

*   **Description:** A roaring lion (Aaron Judge), wearing the New York Yankees #99 jersey, swings a bat with powerful force, launching a ball out of the park.
*   **AI Prompt:**  "A 3D rendered, Pixar-style animation scene. A dynamic close-up of a powerful, roaring lion with a determined expression. The lion is wearing a New York Yankees uniform with jersey number 99 on the back and is swinging a baseball bat with immense force. The bat connects with the baseball, sending it flying out of the frame. The background is blurred to emphasize the action. The stadium is full of woodland animal spectators. There is a burst of energy and excitement in the scene. The scene is set at Yankee Stadium during the 2024-10-30 game."

**Scene 3: Chisholm Jr.'s Homer**

*   **Description:**  A leaping cheetah (Jazz Chisholm Jr.), wearing the New York Yankees #2 jersey, is shown leaping as he connects with the ball for a home run.
*    **AI Prompt:**  "A 3D rendered, Pixar-style animation scene.  A dynamic view of a cheetah with a determined expression, soaring through the air. The cheetah is wearing a New York Yankees uniform with jersey number 2 on the back, and has just connected with a baseball, sending it out of the park. His legs are tucked and arms are outstretched. The background has a blurred stadium full of woodland animal spectators. The scene is full of excitement, set at Yankee Stadium during the 2024-10-30 game."

**Scene 4: Second Inning Single**

*    **Description:** A fox, (Yankee player), slides into third base. He is wearing a NY Yankee uniform with number 32 on his back.
*    **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A dynamic close-up of a leaping fox wearing a New York Yankees uniform with jersey number 32 on his back. He is shown sliding into third base. The background shows a baseball field, with the other two bases blurred in the background. The stands are full of woodland animals, cheering the fox on.  The scene is set at Yankee Stadium during the 2024-10-30 game."

**Scene 5: Stanton's Solo Homer**

*   **Description:**  A powerful bear (Giancarlo Stanton) is shown hitting a baseball for a solo home run, his body tensed, and bat extended.
*   **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A dynamic shot of a hulking bear, looking determined, wearing a New York Yankees uniform with jersey number 27 on the back. He has just hit a baseball with great force, sending it flying into the sky. His body is tensed as he connects with the ball. The background is a blurred stadium filled with excited woodland animal spectators. The scene should convey power and excitement, set at Yankee Stadium on 2024-10-30."

**Scene 6: Away Team Strikeout**

*   **Description:**  A frustrated coyote (Dodger player) is shown striking out on a swing, looking dismayed.
*    **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A medium shot of a frustrated coyote, wearing a Los Angeles Dodgers uniform with jersey number 10 on the back, shown mid-swing, having just missed a baseball. He has a look of dismay on his face, having just struck out. The background shows a blurred batter's box and a crowd of woodland animals. The scene is set at Yankee Stadium during the 2024-10-30 game."

**Scene 7: Away Team Single & Scoring Run**

*   **Description:** A fast rabbit (Dodger player), wearing a Los Angeles Dodgers uniform #30, is shown sprinting and crossing home plate.
*   **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A wide shot of a small rabbit, wearing a Los Angeles Dodgers uniform with jersey number 30 on the back. The rabbit is shown sprinting and crossing home plate. His face shows determination. The background is a baseball field, with other woodland animal players and fans. The scene is set at Yankee Stadium during the 2024-10-30 game. This scene should be full of energy and action."

**Scene 8: Another Away Team Single & Scoring**

*   **Description:** Two raccoons (Dodger players), wearing #4 and #5 Los Angeles Dodgers uniforms, are being shown sprinting into home plate.
*   **AI Prompt:**  "A 3D rendered, Pixar-style animation scene. A dynamic shot showing two raccoons, wearing Los Angeles Dodgers uniforms with jersey numbers 4 and 5 on their backs, running from third base to home. Their faces show determination as they are sprinting. The background includes a partially blurred baseball field with other woodland animal players and fans, cheering. The scene is full of movement, set at Yankee Stadium during the 2024-10-30 game."

**Scene 9: Tying Double**

*    **Description:** A skunk (Dodger player) wearing #17 is connecting with a baseball for a double and is running toward first base.
*    **AI Prompt:**  "A 3D rendered, Pixar-style animation scene. A dynamic action shot showing a skunk, wearing a Los Angeles Dodgers uniform with jersey number 17 on the back. The skunk is swinging a bat and connecting with the ball for a double. His face shows fierce concentration. His body shows movement running toward first base. The background shows a partially blurred baseball field with other woodland animals. The scene is filled with anticipation, set at Yankee Stadium during the 2024-10-30 game."

**Scene 10: Home Team Sacrifice Fly**

*    **Description:** A bat (Yankee player) wearing #15 on the back is shown hitting a sacrifice fly into the outfield. He is shown in mid-flight, wings stretched. A raccoon (Yankee player) wearing #7 is shown scoring a run.
*    **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A wide shot showing a bat, wearing a New York Yankees uniform with jersey number 15 on the back, shown in mid-flight, his wings outstretched. He has just hit a fly ball to the outfield. In the background, a raccoon, wearing a New York Yankees uniform with jersey number 7 on the back, is shown sprinting and scoring a run. The background shows a baseball field with the blurred figures of other woodland animal players and fans. The scene has a sense of action and teamwork, set at Yankee Stadium during the 2024-10-30 game."

**Scene 11: Away Team Sacrifice Fly Tie**

*    **Description:** A swift hawk (Dodger player) wearing #23 is shown flying out to the outfield for a sacrifice fly. Another fast rabbit (Dodger player), wearing #30 is shown scoring.
*    **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A dynamic shot showing a hawk, wearing a Los Angeles Dodgers uniform with jersey number 23 on the back. The hawk is shown in mid-flight, wings outstretched as he hits a sacrifice fly to the outfield. A rabbit, wearing a Los Angeles Dodgers uniform with jersey number 30 on the back, is shown sprinting and sliding into home, scoring a run.  The background shows a baseball field and a blurred crowd of woodland animal spectators. The scene is full of action, set at Yankee Stadium during the 2024-10-30 game."

**Scene 12: Away Team Lead Sacrifice Fly**

*   **Description:** Another swift hawk (Dodger player), wearing a #25 on his back, is shown flying out for another sacrifice fly.
*   **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A dynamic shot showing a hawk, wearing a Los Angeles Dodgers uniform with jersey number 25 on the back. The hawk is shown mid-flight, his wings outstretched, hitting a sacrifice fly to the outfield. A different Dodger player is shown running home and scoring in the background. The background shows a baseball field, with other blurred figures of woodland animal players and fans. The scene is tense, set at Yankee Stadium during the 2024-10-30 game."

**Scene 13: Judge's Double**

*   **Description:** The lion (Aaron Judge) is shown hitting a line drive, wearing #99, running to second base.
*   **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A close up view of a lion, wearing a New York Yankees uniform with jersey number 99 on the back, hitting a line drive for a double, with intense focus. The background includes a blurred view of first base and the outfield.  The stadium is full of woodland animal spectators. The scene is full of energy, set at Yankee Stadium during the 2024-10-30 game."

**Scene 14: Final Out**

*   **Description:** A somber raccoon (Yankee player) wearing #8 on his back is shown striking out at the plate.
*   **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A medium shot of a somber raccoon, wearing a New York Yankees uniform with jersey number 8 on his back, shown striking out on a swing, with a look of disappointment. He has just struck out.  The background is blurred and includes other woodland animals. The scene is full of tension, set at Yankee Stadium during the 2024-10-30 game."

**Scene 15: Away Team Celebration**

*    **Description:** A group of joyous and victorious Dodgers players are shown leaping and running around in celebration. They are comprised of a skunk, rabbit, two hawks, and a coyote. The skunk is wearing #17, the rabbit is wearing #30, the hawks are wearing #23 and #25 and the coyote is wearing #10.
*    **AI Prompt:** "A 3D rendered, Pixar-style animation scene. A wide shot showing a group of joyful and victorious Dodgers players – a skunk, rabbit, two hawks, and a coyote.  They are wearing their respective jerseys: #17, #30, #23, #25, and #10. They are jumping and running around in celebration. The background includes the blurred outline of the baseball field and woodland animal spectators. The scene is full of joy and victory, set at Yankee Stadium during the 2024-10-30 game."

These detailed prompts should help generate images that capture the excitement of the game in a fun and visually engaging way.'



### Game Story Test 3

#### Game Story
The game started with the away team’s big hitters, Ohtani, Betts, and Freeman, all getting out in the first inning.  The home team’s turn was more exciting!  After a walk, Aaron Judge smashed a home run, scoring two runs!  Then, Chisholm Jr. hit another home run, making it 3-0!  The away team struggled, and the home team added another run in the second, making it 4-0 after Verdugo singled in a runner from third.  It seemed like the home team was on a roll.  The away team was down but not out, and in the 5th inning, things got wild!  They loaded the bases, and then Betts singled to score a run. Freeman followed with a single that scored two more. Just like that, the away team were down only one run!  Then, Hernandez hit a double to tie the game up at 5!  The game was tied!  The home team quickly got one run back in the 6th to regain the lead 6-5.  The away team weren't done, though.  In the 8th, they tied the game when Lux hit a sacrifice fly. Then, thanks to some errors and another sacrifice fly by Betts, they took the lead! It was now 7-6 for the away team. The home team tried hard to get back but got close, but ultimately lost. It was a seesaw battle to the very end, with the away team coming back to win it 7-6!

**Scene Descriptions**

1.  **Opening Strikeouts:**  Wide shot of a brightly lit baseball field. The away team’s dugout (on left) shows a disappointed Lion (Ohtani #17), a Badger (Betts #50), and a Bear (Freeman #5), looking frustrated, while a Squirrel (Home team) is celebrating on the base path.
2.  **Judge’s Homerun:** Close-up on a large Muscular Beaver (Judge, #99) swinging mightily with his baseball bat, bat about to connect with the ball, sending it flying. The home team dugout (on right) is full of cheering squirrels and other small woodland animals, while three smaller animals are running around the bases to make the score 2-0.
3.  **Chisholm Jr.’s Homerun:** A medium shot with an energetic Ferret (Chisholm Jr. #2) leaping as he makes contact with the baseball, bat following through with force.
4.  **Verdugo’s Single:** Medium shot from the side of the field as a Fox (Verdugo, #99) hits a line drive past a diving coyote. A squirrel races from third base to score.
5. **Bases Loaded:** Wide shot showing three bases filled with animals wearing away team (Dodger) uniforms. The Lion (Ohtani #17) is on 3rd, the Badger (Betts #50) is on 2nd, and the Bear (Freeman #5) is on 1st base, with a tense expression, while the home team’s pitcher, a wise old Owl, looks on warily from the mound. 
6. **Betts' Single:** Close up, the badger (Betts) bats the ball to shallow center field with a determined look. The Squirrel trying to catch the ball is running but has no chance.
7. **Freeman's Single:** Medium shot of the Bear (Freeman) hitting the ball past a diving Rabbit infield player, with Badger (Betts) and Lion (Ohtani) both scoring.
8.  **Hernandez's Double:** Medium shot of an elegant Deer (Hernandez #3) smashing a line drive into the outfield, with two players running to score.
9.  **Home Team Regains Lead:** Medium shot of the Beaver (Judge) running, with an excited squirrel pointing towards the plate after crossing it.
10. **Lux's Sacrifice Fly:** Close up of a fox (Lux #9) hitting a fly ball that goes far enough for a sacrifice out, but an away team player is seen running home to tie.
11. **Betts' Sacrifice Fly:** Medium shot of the Badger (Betts) hitting a sacrifice fly, with a player running home to take the lead for the away team.
12. **Final Out:** Wide shot of the Beaver (Judge) at bat, with a look of desperation on his face as he swings and misses. The away team (Dodger) players, the Badger (Betts), Lion (Ohtani), and Bear (Freeman), are celebrating joyfully on the field.

**AI Prompts**

**Note:** All prompts will include these base instructions:
*   **Style:** “3D rendered, Pixar-style animated kids film look”
*   **Background People:** “Background people are replaced with woodland animal characters”
*   **General:** “vibrant colors, dynamic poses, exciting action”

**Prompt 1: Opening Strikeouts**

*   **Specifics:** "A wide shot of a brightly lit baseball field. On the left, the away team dugout shows a disappointed Lion wearing a blue Los Angeles Dodgers uniform with jersey number 17 (Ohtani #17), a Badger wearing a blue Los Angeles Dodgers uniform with jersey number 50 (Betts #50), and a Bear wearing a blue Los Angeles Dodgers uniform with jersey number 5 (Freeman #5), looking frustrated. On the basepath, a home team Squirrel wearing a white New York Yankees uniform with number 2 is celebrating. The baseball field has a green field and brown dirt."

**Prompt 2: Judge’s Homerun**

*   **Specifics:** "A close-up on a large Muscular Beaver wearing a white New York Yankees uniform with jersey number 99 (Judge #99), swinging mightily with his baseball bat, bat about to connect with the ball, sending it flying. In the background on the right, the home team dugout is full of cheering squirrels and other small woodland animals in white New York Yankees uniforms. Three smaller woodland animals are running the bases to make the score 2-0."

**Prompt 3: Chisholm Jr.’s Homerun**

*   **Specifics:** "A medium shot with an energetic Ferret wearing a white New York Yankees uniform with jersey number 2 (Chisholm Jr. #2) leaping as he makes contact with the baseball, bat following through with force. The home team (New York Yankees) dugout is in the background, with woodland animals cheering."

**Prompt 4: Verdugo’s Single**

*   **Specifics:** "A medium shot from the side of the field as a Fox wearing a white New York Yankees uniform with jersey number 99 (Verdugo, #99) hits a line drive past a diving Coyote wearing a blue Los Angeles Dodgers uniform. A squirrel races from third base to score. The baseball field has a green field and brown dirt."

**Prompt 5: Bases Loaded**

*   **Specifics:** "A wide shot showing three bases filled with animals wearing blue Los Angeles Dodgers uniforms. The Lion is on 3rd (Ohtani #17), the Badger is on 2nd (Betts #50), and the Bear is on 1st base (Freeman #5), all with tense expressions. The home team’s pitcher, a wise old Owl wearing a white New York Yankees uniform, looks on warily from the mound."

**Prompt 6: Betts' Single**

*   **Specifics:** “A close up of a Badger wearing a blue Los Angeles Dodgers uniform with jersey number 50 (Betts) batting the ball to shallow center field with a determined look. In the background, a Squirrel wearing a white New York Yankees uniform is running but has no chance to catch it.”

**Prompt 7: Freeman's Single**

*   **Specifics:** “A medium shot of the Bear wearing a blue Los Angeles Dodgers uniform with jersey number 5 (Freeman) hitting the ball past a diving Rabbit infield player wearing a white New York Yankees uniform. Badger (Betts #50) and Lion (Ohtani #17) are both running from base to base to score. The baseball field has a green field and brown dirt.”

**Prompt 8: Hernandez's Double**

*   **Specifics:** “A medium shot of an elegant Deer wearing a blue Los Angeles Dodgers uniform with jersey number 3 (Hernandez #3) smashing a line drive into the outfield. Two players wearing blue Los Angeles Dodgers uniforms are running to score from base to base.”

**Prompt 9: Home Team Regains Lead**

*   **Specifics:** "Medium shot of the Beaver (Judge #99), in his white New York Yankees uniform, running, with an excited squirrel in a white New York Yankees uniform pointing towards the plate after crossing it."

**Prompt 10: Lux's Sacrifice Fly**

*  **Specifics**: “Close up of a Fox wearing a blue Los Angeles Dodgers uniform with jersey number 9 (Lux #9) hitting a fly ball that goes far enough for a sacrifice out. An away team player in a blue Los Angeles Dodgers uniform is seen running home to tie.”

**Prompt 11: Betts' Sacrifice Fly**

*  **Specifics**: “Medium shot of a Badger wearing a blue Los Angeles Dodgers uniform with jersey number 50 (Betts) hitting a sacrifice fly, with a player in a blue Los Angeles Dodgers uniform running home to take the lead for the away team. The baseball field has a green field and brown dirt.”

**Prompt 12: Final Out**

*   **Specifics:** "Wide shot of the Beaver wearing a white New York Yankees uniform with jersey number 99 (Judge #99) at bat, with a look of desperation on his face as he swings and misses. The away team (Los Angeles Dodgers) players, the Badger (Betts #50), Lion (Ohtani #17), and Bear (Freeman #5), are celebrating joyfully on the field, all in their blue Los Angeles Dodgers uniforms. The baseball field has a green field and brown dirt."

### Game Story Test 2

**Game:** (Let's assume a fictional game for this example. Feel free to provide specifics if needed. Let's say it's Game 7 of the World Series between the New York Yankees and the Los Angeles Dodgers)

**Scene Descriptions:**

1.  **Opening - The Roar of the Crowd:** A wide shot of a packed stadium at dusk. Instead of human spectators, there's a diverse crowd of woodland creatures (squirrels, owls, foxes, raccoons, deer etc.) filling the stands. The atmosphere is electric. The Yankee dugout (left) and Dodger dugout (right) are visible and the animals representing the respective teams are looking serious and ready for the game.

2.  **Pitcher on the Mound:** A close-up shot of the Yankees' starting pitcher, a powerful-looking Lion with a focused gaze, ready to deliver a pitch. He is wearing the classic Yankee pinstripes and has jersey number 22 on his back. In the batter's box, facing him, is a focused, determined looking bulldog in the Dodgers uniform.

3.  **Batting Power:** The bulldog swings hard, connecting with the ball. The camera is angled up, showcasing the intense swing. The background is slightly blurred with the focus being on the bulldog and bat.

4.  **Catch at the Wall:** A high-flying catch by the Yankee outfielder, a swift hawk with jersey number 7, who leaps over a short fence in front of the woodland creatures in the stands, bringing the ball into his mitt just before colliding with the wall.

5.  **Base Stealing Fox:** A nimble fox with jersey number 14, representing the Dodgers, steals second base, sliding in under the tag of the second baseman (a stoic bear with jersey number 4). The slide kicks up a cloud of dust.

6. **Home Run Celebration:** The Dodgers' Batter hits a monster home run. The woodland creatures in the stands cheer as the fox rounds the bases in jubilation and high-fives the team in the dugout.

7.  **Defensive Stand:** Intense action at home plate. A play at the plate where a Dodger baserunner (the aforementioned fox) is tagged out with a close slide by the Yankee Catcher (an agile badger with jersey number 28).

8.  **Yankee Rally:** Multiple Yankee hits lead to a bases-loaded situation. The scene has tension and excitement. A close-up shot of the lion with jersey number 22 is shown from below as he looks toward the plate, determination in his eyes.

9.  **The Grand Slam:** The Lion with jersey number 22 connects with a pitch, hitting a grand slam. The reaction of the Yankee woodland animal crowd is one of pure joy.

10. **Final Out:** A dramatic final out, a pop-up caught by the hawk in centerfield ends the game. A wide shot of the field with the Yankees celebrating, and the Dodgers looking downcast.

11.  **Victory Celebration:** The Yankees team is shown high fiving and celebrating their World Series win on the field. Confetti falls from above. The Lion stands near the center of the huddle holding the World Series Trophy high.

**AI Prompts:**

**General Settings for all Prompts:** 3D Rendered Pixar-style, animated kids film look, vibrant colors, dynamic poses, exciting action, no humans present.

**Prompt 1: The Roar of the Crowd**
*   **Description:** A wide shot of a packed baseball stadium at dusk, filled with woodland creatures such as squirrels, owls, foxes, raccoons, and deer as fans. The atmosphere is electric. The Yankee dugout (left) is visible with the Lion (22) looking focused with his team. The Dodger dugout (right) shows the bulldog and his team also focused and ready to play.
*   **Details:**  Use vibrant colors for the stadium seats, showing the team colors in the fans outfits. Soft evening lighting. Include the stadium lights around the perimeter of the field.

**Prompt 2: Pitcher on the Mound**
*   **Description:** Close up, a powerful Lion with a focused gaze, wearing New York Yankee pinstripes with jersey number 22 on his back, ready to deliver a pitch from the pitcher's mound. A bulldog in a Dodger uniform waits in the batter's box.
*   **Details:** Focus on the determined expression on the Lion's face. The pitcher's mound and batter's box should be clearly visible. Dodger logo visible on the bulldog uniform.

**Prompt 3: Batting Power**
*   **Description:** Dynamic angle up showing the bulldog in Dodgers uniform swings hard, making contact with the baseball bat.
*   **Details:** Emphasize the power of the swing. Background should be slightly blurred to emphasize movement.

**Prompt 4: Catch at the Wall**
*   **Description:** A swift hawk with jersey number 7, wearing a New York Yankee uniform, leaping over a short fence in front of the woodland creature fans in the stands, making a diving catch at the outfield wall.
*   **Details:** Show the dynamic movement of the hawk in mid-air, making the catch just before colliding with the wall. The hawk should be fully extended.

**Prompt 5: Base Stealing Fox**
*   **Description:**  A nimble fox wearing a Los Angeles Dodgers uniform with jersey number 14, sliding into second base under the tag of a stoic bear in a New York Yankees uniform with jersey number 4. Dust from the slide fills the air.
*   **Details:** Show the speed and agility of the fox. Capture the intensity of the slide. Clear positioning of the players on the bases.

**Prompt 6: Home Run Celebration**
*   **Description:** The bulldog batter just hit a home run and is rounding the bases as the Dodgers woodland animal fans erupt. He is high-fiving his teammates in the dugout.
*   **Details:** Show the celebration by all the players in the Dodger dugout. Woodland creature fans in the background cheering.

**Prompt 7: Defensive Stand**
*   **Description:** Intense action at home plate. The fox in a Dodgers uniform sliding in to home plate, getting tagged out by the badger in a New York Yankees uniform with jersey number 28.
*   **Details:** Create a sense of the close play with the animals moving quickly.

**Prompt 8: Yankee Rally**
*    **Description:** The bases are loaded with the New York Yankees. The Lion is shown in a low-angle shot, looking towards the plate with determination on his face. He is wearing New York Yankee pinstripes with jersey number 22.
*   **Details:** Use the low angle to emphasize the power of the Lion character. The baseball diamond is visible in the background.

**Prompt 9: The Grand Slam**
*   **Description:** The Lion (jersey 22) connects with a pitch, hitting a grand slam. The Yankee woodland animal crowd is cheering in the background.
*   **Details:** Show a powerful hit with an emphasis on the ball leaving the bat. The background should be full of movement.

**Prompt 10: Final Out**
*   **Description:** A pop-up high in the air being caught by the hawk in centerfield (jersey 7). This concludes the game. The New York Yankee players are celebrating as the Los Angeles Dodgers show disappointment on the field.
*   **Details:** Show the celebration from the New York Yankees and the disappointment from the Los Angeles Dodgers. The action is shown on the field.

**Prompt 11: Victory Celebration**
*   **Description:** The New York Yankees team is celebrating their World Series victory on the field. Confetti is falling from the sky and the Lion (jersey 22) is holding the trophy in the center of the huddle.
*   **Details:** Show the entire team celebrating. Focus on the joy and elation.

### Game Story Test

The game started with the visiting team's batters, Ohtani, Betts, and Freeman, flying and grounding out against pitcher Cole. In the bottom of the first, Torres grounded out. Then, Soto walked, and the crowd roared as Judge hit a huge home run, scoring Soto! Next, Chisholm Jr. also homered! The home team now had a 3-0 lead, but Stanton, Rizzo, and the team all were unable to further the lead.

The second inning saw the visiting team again go down without any scoring. In the bottom of the second, Volpe doubled, advancing to third on a groundout. Verdugo then singled, driving Volpe in. After the hit, the bases were loaded from walks, but Chisholm Jr. grounded out to end the inning. 

The top of the third saw the visiting team struggle again, with a series of outs, and a single walk. In the bottom of the third, the home team added to their lead as Stanton hit another home run, making it 5-0. Rizzo, Volpe, and Wells were then retired to end the inning.

The fourth inning was frustrating for the visiting team, as a walk led to a single, but they couldn't bring any of their runners home. In the bottom of the fourth, the home team loaded the bases again, but couldn't score.

The fifth inning was a wild one. The visiting team loaded the bases via error, then scored 5 runs on a series of hits, including a double by Hernández, to tie the game. But the home team wasn't ready to give up. In the bottom of the inning they loaded the bases again via hit, hit by pitch, and walk, but couldn't score any runs.

The sixth was frustrating for the visiting team with three outs again. The home team used some walks and a sacrifice fly to take a 6-5 lead. 

The seventh was a mixed bag, with the visiting team getting some more walks, but still not scoring. The home team also failed to score.

The eighth inning saw the visiting team get multiple runners into scoring position and then scored two runs to take the lead for the first time in the game. In the bottom of the inning, the home team threatened again with another double, but didn't get any runs.

In the ninth inning, the visiting team had some more runners on base, but couldn't extend their lead. The home team in their half of the ninth, went down without any success, giving the visiting team a 7-6 win. 

### Storyboard Scenes & Prompts

**Scene 1:**
*   **Description:** Aaron Judge hits a towering home run, with another team player running across the plate. The stands are full of animal characters cheering.
*   **Prompt:** A 3D render of a massive bear, wearing a New York Yankees uniform, swings a large wooden bat, sending a baseball soaring through the air. A rabbit in a Yankees uniform is racing towards the home plate. The baseball field has a packed stadium filled with cheering squirrels, owls, and deer. The scene is set during a bright sunny day with a clear blue sky, using a pixar-style animation.

**Scene 2:**
*   **Description:** Jazz Chisholm Jr. hits another home run, the crowd is going crazy
*  **Prompt:** A 3D render of a small, nimble fox, wearing a New York Yankees uniform, having just hit a baseball out of the park. The stadium is packed with squirrels, badgers, and raccoons celebrating wildly, waving small flags and banners. The scene uses a bright, cheerful pixar-style animation with a clear focus on the fox and the ecstatic crowd.

**Scene 3:**
*   **Description:** Anthony Volpe slides into second for a double, a tense moment in the game.
*   **Prompt:** A 3D render of a weasel, in a New York Yankees uniform, sliding into second base, kicking up dirt. The second baseman, a grumpy looking badger in a Los Angeles Dodgers uniform, is reaching for the ball.  The scene is focused with a shallow depth of field with only a few spectators, composed of woodland creatures, in the background, using a pixar-style animated look.

**Scene 4:**
*   **Description:** A frustrated looking pitcher, Jack Flaherty, looks at the dugout after giving up a big hit.
*    **Prompt:** A 3D render of a disgruntled looking lion, wearing a Los Angeles Dodgers uniform, standing on the mound. He has his back turned to the batter and is facing towards the dugout with an exasperated expression. A few woodland animals in Dodgers uniform are in the dugout with their heads down looking worried. The field is a little blurry to bring the focus to the lion. The style is a 3D rendered pixar-style animated look.

**Scene 5:**
*   **Description:** The visiting team's batters celebrating as the score is tied after a big inning.
*   **Prompt:** A 3D render of a group of different animals wearing Los Angeles Dodgers uniforms, high-fiving and celebrating.  A giraffe, a zebra, and a cheetah, are all jumping in the air celebrating. The stadium background is out of focus and a little blurry.  The scene is a 3D pixar-style animated look.

**Scene 6:**
*   **Description:** Giancarlo Stanton comes up to bat and hits a homerun, getting a big reaction from the crowd
*   **Prompt:** A 3D render of a very large gorilla, wearing a New York Yankees uniform, completing a massive home run swing with a baseball in the air.  A group of woodland animals in Yankees uniform are standing in the dugout with excited expressions. The stadium is packed, and in the background are thousands of cheering animal spectators. The scene is during a bright sunny day with a pixar-style 3d animated look.

**Scene 7:**
*   **Description:** A tense moment with a runner on third and a hit batter.
*   **Prompt:** A 3D render of a tense scene on a baseball field. A skunk in a New York Yankees uniform is standing on third base, with a squirrel at second after being hit by a pitch. A worried looking raccoon is standing at the plate. The pitcher is a bored looking porcupine in a Los Angeles Dodgers uniform. A few woodland creatures are in the blurry background.  The style is a pixar-style animation.

**Scene 8:**
*   **Description:** A home team player slides into home, scoring an important run.
*   **Prompt:** A 3D render of a weasel wearing a New York Yankees uniform, sliding into home plate, a cloud of dust billowing around him. The catcher, a stoic looking bear in a Los Angeles Dodgers uniform, is attempting to make the tag, but is too late.  The crowd is filled with various woodland creatures like rabbits, skunks, and opossums cheering from the stands. The style should be a 3D pixar-style animation with a shallow depth of field to bring focus to the player.

**Scene 9:**
*   **Description:** Teoscar Hernández hits a clutch double late in the game.
*    **Prompt:** A 3D render of a graceful leopard wearing a Los Angeles Dodgers uniform, completing a powerful swing that sends a baseball flying. Two other animals, also in Dodgers uniform, are rounding the bases. The background features a packed stadium with a crowd of various woodland creatures like owls, foxes, and deer. The scene is rendered with bright sunlight and a dynamic sense of movement, in a pixar-style animated look.

**Scene 10:**
*   **Description:** The home team's dugout after the final out of the game.
*   **Prompt:** A 3D render of a sad group of animals wearing New York Yankees uniforms. They are sitting in the dugout with their heads in their hands, and look defeated. The style is a 3D pixar-style animation with soft lighting.
